In [1]:
import random
import copy
from datetime import datetime, timedelta
from IPython.display import display, HTML
import ipywidgets as widgets

# بيانات الفرق والملاعب والأوقات
teams = [
    "Al Ahly", "Zamalek", "Pyramids", "Masry", "Future", "Ismaily",
    "Smouha", "ENPPI", "Ceramica", "National Bank", "Talaea El Gaish",
    "Alexandria Union", "El Dakhleya", "El Gouna", "Zed",
    "Modern Sport", "Pharco", "Wadi Degla"
]

venues = [
    "Cairo Stadium", "Borg El Arab", "Air Defense Stadium",
    "Suez Stadium", "Alexandria Stadium", "Petro Sport Stadium",
    "Military Academy Stadium", "Al Salam Stadium", "El Sekka El Hadeed Stadium",
    "Zed Club Stadium"
]

match_times = ["17:00", "20:00"]

In [2]:
# مدة البطولة
start_date = datetime(2025, 5, 1)
end_date = datetime(2026, 1, 31)

# إعداد أيام البطولة
all_dates = []
current_date = start_date
while current_date <= end_date:
    all_dates.append(current_date)
    current_date += timedelta(days=1)

# تعريف الـ Match
class Match:
    def __init__(self, team1, team2, date, time, venue):
        self.team1 = team1
        self.team2 = team2
        self.date = date
        self.time = time
        self.venue = venue

    def __repr__(self):
        return f"{self.team1} vs {self.team2} on {self.date.strftime('%Y-%m-%d')} at {self.time} in {self.venue}"

# توليد جدول دوري أسبوعي مع تكرار المباريات مرتين
def generate_weekly_schedule(teams, venues, all_dates, match_times, min_rest_days=4):
    schedule = []
    last_played = {team: start_date - timedelta(days=min_rest_days) for team in teams}
    day_info = {d: {'count': 0, 'venues': set(), 'times': set()} for d in all_dates}
    pairings = [(t1, t2) for t1 in teams for t2 in teams if t1 != t2]
    random.shuffle(pairings)
    matches_count = {(t1, t2): 0 for t1 in teams for t2 in teams if t1 != t2}
    week_dates = all_dates[::7]

    for week_start in week_dates:
        used_teams = set()
        for home, away in pairings:
            if matches_count[(home, away)] >= 1:
                continue
            if home in used_teams or away in used_teams:
                continue
            possible_days = [d for d in all_dates if week_start <= d < week_start + timedelta(days=7)
                             and (d - last_played[home]).days >= min_rest_days
                             and (d - last_played[away]).days >= min_rest_days
                             and day_info[d]['count'] < 2]
            if not possible_days:
                continue
            random.shuffle(possible_days)
            for date in possible_days:
                available_times = [t for t in match_times if t not in day_info[date]['times']]
                available_venues = [v for v in venues if v not in day_info[date]['venues']]
                if not available_times or not available_venues:
                    continue
                time = random.choice(available_times)
                venue = random.choice(available_venues)
                match = Match(home, away, date, time, venue)
                schedule.append(match)
                last_played[home] = date
                last_played[away] = date
                matches_count[(home, away)] += 1
                used_teams.update([home, away])
                day_info[date]['count'] += 1
                day_info[date]['venues'].add(venue)
                day_info[date]['times'].add(time)
                break

    schedule.sort(key=lambda m: m.date)
    return schedule

# طباعة جدول كل فريق
def print_team_schedules(schedule):
    team_schedule = {team: [] for team in teams}
    for match in schedule:
        team_schedule[match.team1].append(match)
        team_schedule[match.team2].append(match)
    for team, matches in team_schedule.items():
        print(f"\n=== Schedule for {team} ===")
        matches.sort(key=lambda m: m.date)
        for m in matches:
            opponent = m.team2 if m.team1 == team else m.team1
            print(f"{m.date.strftime('%Y-%m-%d')} at {m.time} vs {opponent} in {m.venue}")

# توليد عدة احتمالات
num_possibilities = 5
all_possible_schedules = [generate_weekly_schedule(teams, venues, all_dates, match_times) for _ in range(num_possibilities)]

# طباعة أول 5 احتمالات
# for i, schedule in enumerate(all_possible_schedules):
#     print(f"\n=== Schedule {i+1} ===")
#     for m in schedule[:10]:  # أول 10 مباريات لكل جدول
#         print(m)
#     print_team_schedules(schedule)  # جدول كل فريق


In [3]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Dropdown لاختيار الاحتمال
schedule_dropdown = widgets.Dropdown(
    options=[f"Schedule {i+1}" for i in range(num_possibilities)],
    description="Choose:",
    disabled=False
)

# Dropdown لاختيار نوع العرض
view_dropdown = widgets.Dropdown(
    options=["Full League Schedule", "Team Schedule"],
    description="View:",
    disabled=False
)

# Dropdown لاختيار الفريق
team_dropdown = widgets.Dropdown(
    options=teams,
    description="Team:",
    disabled=False
)

# زرار للعرض
button = widgets.Button(description="Show Schedule")

# Output area
output = widgets.Output()

# الدالة المسؤولة عن العرض
def show_schedule(b):
    with output:
        clear_output(wait=True)  # يمسح الجدول القديم قبل العرض
        selected_index = int(schedule_dropdown.value.split()[1]) - 1
        schedule = all_possible_schedules[selected_index]
        view = view_dropdown.value
        if view == "Full League Schedule":
            html = "<h2>Full League Schedule</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Home</th><th>Away</th><th>Venue</th></tr>"
            for match in schedule:
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{match.team1}</td><td>{match.team2}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))
        else:
            team = team_dropdown.value
            team_matches = [m for m in schedule if m.team1 == team or m.team2 == team]
            html = f"<h2>Schedule for {team}</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Opponent</th><th>Venue</th></tr>"
            for match in team_matches:
                opponent = match.team2 if match.team1 == team else match.team1
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{opponent}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))

button.on_click(show_schedule)

# عرض الـ widgets
display(schedule_dropdown, view_dropdown, team_dropdown, button, output)


Dropdown(description='Choose:', options=('Schedule 1', 'Schedule 2', 'Schedule 3', 'Schedule 4', 'Schedule 5')…

Dropdown(description='View:', options=('Full League Schedule', 'Team Schedule'), value='Full League Schedule')

Dropdown(description='Team:', options=('Al Ahly', 'Zamalek', 'Pyramids', 'Masry', 'Future', 'Ismaily', 'Smouha…

Button(description='Show Schedule', style=ButtonStyle())

Output()

In [4]:
def compute_fitness_verbose(schedule, min_rest_days=4, weights=None):
    if weights is None:
        weights = {'venue_conflict': 1.0, 'rest_violation': 0.7, 'repeated_opponent': 0.0, 'time_balance': 0.3}

    penalty = 0
    venue_day_time = {}
    last_played = {}
    matches_per_pair = {}
    team_times = {}

    venue_conflict_count = 0
    rest_violation_count = 0
    repeated_opponents_count = 0
    time_balance_penalty = 0

    for match in schedule:
        key = (match.date, match.time, match.venue)
        if key in venue_day_time:
            penalty += weights['venue_conflict']
            venue_conflict_count += 1
        else:
            venue_day_time[key] = True

        for team in [match.team1, match.team2]:
            if team in last_played:
                delta_days = (match.date - last_played[team]).days
                if delta_days < min_rest_days:
                    penalty += weights['rest_violation']
                    rest_violation_count += 1
            last_played[team] = match.date

        pair = tuple(sorted([match.team1, match.team2]))
        if pair in matches_per_pair:
            penalty += weights['repeated_opponent']
            repeated_opponents_count += 1
        matches_per_pair[pair] = matches_per_pair.get(pair,0) + 1

        for team in [match.team1, match.team2]:
            if team not in team_times:
                team_times[team] = {}
            team_times[team][match.time] = team_times[team].get(match.time,0) + 1

    for team, times in team_times.items():
        if len(times) > 1:
            diff = abs(times.get('17:00',0) - times.get('20:00',0))
            penalty += diff * weights['time_balance']
            time_balance_penalty += diff

    fitness = max(0, 100 - penalty)

    print(f"Fitness: {fitness}")
    print(f"Venue conflicts: {venue_conflict_count}, Rest violations: {rest_violation_count}, Repeated opponents: {repeated_opponents_count}, Time balance penalty: {time_balance_penalty}")

    return fitness



In [5]:
best_score = -1
best_schedule = None

for schedule in all_possible_schedules:
    score = compute_fitness_verbose(schedule)
    if score > best_score:
        best_score = score
        best_schedule = schedule

print(f"Best fitness: {best_score}")


Fitness: 67.6
Venue conflicts: 0, Rest violations: 0, Repeated opponents: 153, Time balance penalty: 108
Fitness: 68.80000000000001
Venue conflicts: 0, Rest violations: 0, Repeated opponents: 153, Time balance penalty: 104
Fitness: 73.6
Venue conflicts: 0, Rest violations: 0, Repeated opponents: 153, Time balance penalty: 88
Fitness: 71.2
Venue conflicts: 0, Rest violations: 0, Repeated opponents: 153, Time balance penalty: 96
Fitness: 76.0
Venue conflicts: 0, Rest violations: 0, Repeated opponents: 153, Time balance penalty: 80
Best fitness: 76.0


In [6]:
def single_point_crossover(parent1, parent2):
    """
    Modified single point crossover for sports scheduling
    Returns: (child_schedule, crossover_info)
    """
    # 1. Handle length differences
    min_len = min(len(parent1), len(parent2))
    
    # Track crossover information for GUI
    crossover_info = {
        'parent1_length': len(parent1),
        'parent2_length': len(parent2),
        'crossover_point': None,
        'matches_from_parent1': 0,
        'matches_from_parent2': 0,
        'added_matches': 0,
        'child_length': 0
    }
    
    if min_len < 2:
        # If schedule is too short, return a parent
        selected_parent = parent1 if random.random() < 0.5 else parent2
        child_matches = selected_parent.copy()
        crossover_info['child_length'] = len(child_matches)
        crossover_info['note'] = 'Returned parent (schedule too short)'
        return child_matches, crossover_info
    
    # 2. Select crossover point
    point = random.randint(1, min_len - 1)
    crossover_info['crossover_point'] = point
    
    # 3. Build child while preserving data
    child_matches = []
    
    # Take matches from parent1 up to crossover point
    for i in range(point):
        if i < len(parent1):
            child_matches.append(copy.deepcopy(parent1[i]))
            crossover_info['matches_from_parent1'] += 1
    
    # Take matches from parent2 after crossover point
    for i in range(point, len(parent2)):
        if i < len(parent2):
            # Check for duplicate matches
            match_exists = False
            for existing_match in child_matches:
                if (existing_match.team1 == parent2[i].team1 and 
                    existing_match.team2 == parent2[i].team2):
                    match_exists = True
                    break
            
            if not match_exists:
                child_matches.append(copy.deepcopy(parent2[i]))
                crossover_info['matches_from_parent2'] += 1
    
    # 4. If child is too short, add additional matches from parents
    if len(child_matches) < min_len:
        # Collect all unique matches from both parents
        all_matches = []
        seen_pairs = set()
        
        for match in parent1 + parent2:
            pair_key = f"{match.team1}_{match.team2}"
            if pair_key not in seen_pairs:
                seen_pairs.add(pair_key)
                all_matches.append(copy.deepcopy(match))
        
        # Take remaining needed matches
        remaining_needed = min_len - len(child_matches)
        random.shuffle(all_matches)
        
        for match in all_matches:
            if len(child_matches) >= min_len:
                break
                
            # Check for duplicates in child
            match_exists = False
            for existing in child_matches:
                if (existing.team1 == match.team1 and 
                    existing.team2 == match.team2):
                    match_exists = True
                    break
            
            if not match_exists:
                child_matches.append(match)
                crossover_info['added_matches'] += 1
    
    crossover_info['child_length'] = len(child_matches)
    crossover_info['note'] = 'Crossover completed successfully'
    
    return child_matches, crossover_info

In [7]:
def improved_ensure_min_matches(child_matches, parent1, parent2, target_length):
    """
    Improved version of ensure_min_matches function
    Returns: (updated_child_matches, operation_info)
    """
    operation_info = {
        'initial_length': len(child_matches),
        'target_length': target_length,
        'added_from_parents': 0,
        'final_length': 0,
        'operation_type': 'ensure_min_matches'
    }
    
    if len(child_matches) >= target_length:
        operation_info['final_length'] = len(child_matches)
        operation_info['note'] = 'No matches added (already at target)'
        return child_matches, operation_info
    
    # 1. First: Add all unique matches from parents
    all_unique_from_parents = []
    seen_pairs = set()
    
    # Matches already in child
    for match in child_matches:
        pair_key = f"{match.team1}_{match.team2}_{match.date}"
        seen_pairs.add(pair_key)
    
    # Collect all matches from parents
    all_parent_matches = []
    for parent in [parent1, parent2]:
        for match in parent:
            pair_key = f"{match.team1}_{match.team2}_{match.date}"
            if pair_key not in seen_pairs:
                seen_pairs.add(pair_key)
                all_parent_matches.append(copy.deepcopy(match))
    
    # Shuffle and add
    random.shuffle(all_parent_matches)
    
    # 2. Add until reaching target length
    matches_added = 0
    while len(child_matches) < target_length and all_parent_matches:
        child_matches.append(all_parent_matches.pop())
        matches_added += 1
    
    operation_info['added_from_parents'] = matches_added
    
    # 3. If still missing, add new matches
    if len(child_matches) < target_length:
        remaining = target_length - len(child_matches)
        operation_info['remaining_needed'] = remaining
        operation_info['note'] = f'Still need {remaining} matches (could generate new ones)'
    else:
        operation_info['remaining_needed'] = 0
        operation_info['note'] = 'Target length reached successfully'
    
    operation_info['final_length'] = len(child_matches)
    
    return child_matches, operation_info

In [8]:
def two_point_crossover(parent1, parent2):
    """
    Modified two-point crossover for sports scheduling
    Returns: (child_schedule, crossover_info)
    """
    # Track crossover information for GUI
    crossover_info = {
        'parent1_length': len(parent1),
        'parent2_length': len(parent2),
        'crossover_point1': None,
        'crossover_point2': None,
        'matches_from_parent1': 0,
        'matches_from_parent2': 0,
        'unique_matches_added': 0,
        'child_length': 0,
        'operation_type': 'two_point_crossover'
    }
    
    # 1. Handle length differences
    min_len = min(len(parent1), len(parent2))
    
    if min_len < 3:  # Need at least 3 for two points
        # Fall back to single point crossover
        child_matches, single_point_info = single_point_crossover(parent1, parent2)
        crossover_info['note'] = 'Used single point crossover (min_len < 3)'
        crossover_info['fallback_to_single_point'] = True
        crossover_info.update(single_point_info)
        crossover_info['final_length'] = len(child_matches)
        return child_matches, crossover_info
    
    # 2. Select two crossover points
    point1, point2 = sorted(random.sample(range(1, min_len), 2))
    crossover_info['crossover_point1'] = point1
    crossover_info['crossover_point2'] = point2
    
    # 3. Build child
    child_matches = []
    seen_pairs = set()
    
    # Part from parent 1 (before point1)
    for i in range(point1):
        if i < len(parent1):
            match = copy.deepcopy(parent1[i])
            pair_key = f"{match.team1}_{match.team2}"
            if pair_key not in seen_pairs:
                seen_pairs.add(pair_key)
                child_matches.append(match)
                crossover_info['matches_from_parent1'] += 1
                crossover_info['unique_matches_added'] += 1
    
    # Part from parent 2 (between point1 and point2)
    for i in range(point1, point2):
        if i < len(parent2):
            match = copy.deepcopy(parent2[i])
            pair_key = f"{match.team1}_{match.team2}"
            if pair_key not in seen_pairs:
                seen_pairs.add(pair_key)
                child_matches.append(match)
                crossover_info['matches_from_parent2'] += 1
                crossover_info['unique_matches_added'] += 1
    
    # Part from parent 1 (after point2)
    for i in range(point2, len(parent1)):
        if i < len(parent1):
            match = copy.deepcopy(parent1[i])
            pair_key = f"{match.team1}_{match.team2}"
            if pair_key not in seen_pairs:
                seen_pairs.add(pair_key)
                child_matches.append(match)
                crossover_info['matches_from_parent1'] += 1
                crossover_info['unique_matches_added'] += 1
    
    # 4. Ensure minimum number of matches
    child_matches, ensure_info = improved_ensure_min_matches(child_matches, parent1, parent2, min_len)
    
    crossover_info['child_length'] = len(child_matches)
    crossover_info['ensure_min_matches_info'] = ensure_info
    
    # Update counts after ensure_min_matches
    if 'added_from_parents' in ensure_info:
        additional_from_parents = ensure_info['added_from_parents']
        if additional_from_parents > 0:
            # Estimate distribution (half from each parent)
            crossover_info['matches_from_parent1'] += additional_from_parents // 2
            crossover_info['matches_from_parent2'] += additional_from_parents - (additional_from_parents // 2)
            crossover_info['unique_matches_added'] += additional_from_parents
    
    crossover_info['note'] = 'Two-point crossover completed successfully'
    
    return child_matches, crossover_info

In [9]:
def uniform_crossover(parent1, parent2):
    """
    Modified uniform crossover for sports scheduling
    Returns: (child_schedule, crossover_info)
    """
    # Track crossover information for GUI
    crossover_info = {
        'parent1_length': len(parent1),
        'parent2_length': len(parent2),
        'matches_from_parent1': 0,
        'matches_from_parent2': 0,
        'matches_from_longer_parent': 0,
        'unique_matches_added': 0,
        'child_length': 0,
        'operation_type': 'uniform_crossover',
        'crossover_probability': 0.5  # 50% chance from each parent
    }
    
    min_len = min(len(parent1), len(parent2))
    max_len = max(len(parent1), len(parent2))
    
    child_matches = []
    seen_pairs = set()
    
    # For first min_len matches
    for i in range(min_len):
        # Randomly select which parent to take from
        if random.random() < 0.5:
            source = parent1[i] if i < len(parent1) else None
            parent_source = 'parent1'
        else:
            source = parent2[i] if i < len(parent2) else None
            parent_source = 'parent2'
        
        if source:
            match = copy.deepcopy(source)
            pair_key = f"{match.team1}_{match.team2}"
            if pair_key not in seen_pairs:
                seen_pairs.add(pair_key)
                child_matches.append(match)
                
                # Track which parent contributed
                if parent_source == 'parent1':
                    crossover_info['matches_from_parent1'] += 1
                else:
                    crossover_info['matches_from_parent2'] += 1
                
                crossover_info['unique_matches_added'] += 1
    
    # For additional matches from longer parent
    if max_len > min_len:
        longer_parent = parent1 if len(parent1) > len(parent2) else parent2
        longer_parent_name = 'parent1' if len(parent1) > len(parent2) else 'parent2'
        
        for i in range(min_len, max_len):
            if i < len(longer_parent):
                match = copy.deepcopy(longer_parent[i])
                pair_key = f"{match.team1}_{match.team2}"
                if pair_key not in seen_pairs:
                    seen_pairs.add(pair_key)
                    child_matches.append(match)
                    
                    # Track longer parent contribution
                    crossover_info['matches_from_longer_parent'] += 1
                    if longer_parent_name == 'parent1':
                        crossover_info['matches_from_parent1'] += 1
                    else:
                        crossover_info['matches_from_parent2'] += 1
                    
                    crossover_info['unique_matches_added'] += 1
    
    # Ensure minimum number of matches
    child_matches, ensure_info = improved_ensure_min_matches(child_matches, parent1, parent2, min_len)
    
    crossover_info['child_length'] = len(child_matches)
    crossover_info['ensure_min_matches_info'] = ensure_info
    
    # Update counts after ensure_min_matches
    if 'added_from_parents' in ensure_info:
        additional_from_parents = ensure_info['added_from_parents']
        if additional_from_parents > 0:
            # Estimate distribution for additional matches
            estimated_from_parent1 = additional_from_parents // 2
            estimated_from_parent2 = additional_from_parents - estimated_from_parent1
            
            crossover_info['matches_from_parent1'] += estimated_from_parent1
            crossover_info['matches_from_parent2'] += estimated_from_parent2
            crossover_info['unique_matches_added'] += additional_from_parents
            crossover_info['additional_matches_added'] = additional_from_parents
    
    # Calculate selection percentages
    total_matches = crossover_info['matches_from_parent1'] + crossover_info['matches_from_parent2']
    if total_matches > 0:
        crossover_info['percent_from_parent1'] = round((crossover_info['matches_from_parent1'] / total_matches) * 100, 1)
        crossover_info['percent_from_parent2'] = round((crossover_info['matches_from_parent2'] / total_matches) * 100, 1)
    
    crossover_info['note'] = 'Uniform crossover completed successfully'
    
    return child_matches, crossover_info

In [10]:
def swap_mutation(schedule, mutation_rate=0.1):
    """
    Swap mutation: exchange two random matches in the schedule
    Returns: (mutated_schedule, mutation_info)
    """
    # Track mutation information for GUI
    mutation_info = {
        'initial_length': len(schedule),
        'mutation_rate': mutation_rate,
        'mutation_applied': False,
        'swapped_indices': None,
        'matches_swapped': 0,
        'final_length': 0,
        'operation_type': 'swap_mutation'
    }
    
    mutated = copy.deepcopy(schedule)
    mutation_info['final_length'] = len(mutated)
    
    if len(mutated) < 2:
        mutation_info['note'] = 'No mutation applied (schedule too short)'
        return mutated, mutation_info
    
    # Check if mutation should be applied
    if random.random() < mutation_rate:
        # Select two different indices
        idx1, idx2 = random.sample(range(len(mutated)), 2)
        
        # Perform the swap
        mutated[idx1], mutated[idx2] = mutated[idx2], mutated[idx1]
        
        # Record mutation details
        mutation_info['mutation_applied'] = True
        mutation_info['swapped_indices'] = (idx1, idx2)
        mutation_info['matches_swapped'] = 2
        
        # Get match details for GUI display
        match1_info = {
            'original_position': idx1,
            'new_position': idx2,
            'teams': f"{mutated[idx2].team1} vs {mutated[idx2].team2}",
            'time': mutated[idx2].time,
            'venue': mutated[idx2].venue
        }
        
        match2_info = {
            'original_position': idx2,
            'new_position': idx1,
            'teams': f"{mutated[idx1].team1} vs {mutated[idx1].team2}",
            'time': mutated[idx1].time,
            'venue': mutated[idx1].venue
        }
        
        mutation_info['match_details'] = {
            'match1': match1_info,
            'match2': match2_info
        }
        
        mutation_info['note'] = f'Swap mutation applied: positions {idx1} and {idx2} exchanged'
    else:
        mutation_info['note'] = 'No mutation applied (random check failed)'
    
    mutation_info['final_length'] = len(mutated)
    
    return mutated, mutation_info

In [11]:
def change_venue_mutation(schedule, venues, mutation_rate=0.1):
    """
    Change venue mutation: change venue of a random match
    Returns: (mutated_schedule, mutation_info)
    """
    # Track mutation information for GUI
    mutation_info = {
        'initial_length': len(schedule),
        'available_venues_count': len(venues),
        'mutation_rate': mutation_rate,
        'mutation_applied': False,
        'match_index': None,
        'old_venue': None,
        'new_venue': None,
        'final_length': 0,
        'operation_type': 'change_venue_mutation'
    }
    
    mutated = copy.deepcopy(schedule)
    mutation_info['final_length'] = len(mutated)
    
    if not mutated or not venues:
        mutation_info['note'] = 'No mutation applied (empty schedule or no venues)'
        return mutated, mutation_info
    
    # Check if mutation should be applied
    if random.random() < mutation_rate:
        # Select a random match
        idx = random.randint(0, len(mutated) - 1)
        old_venue = mutated[idx].venue
        
        # Find available alternative venues
        available_venues = [v for v in venues if v != old_venue]
        
        mutation_info['available_alternative_venues'] = len(available_venues)
        
        if available_venues:
            # Select new venue
            new_venue = random.choice(available_venues)
            
            # Apply the mutation
            mutated[idx].venue = new_venue
            
            # Record mutation details
            mutation_info['mutation_applied'] = True
            mutation_info['match_index'] = idx
            mutation_info['old_venue'] = old_venue
            mutation_info['new_venue'] = new_venue
            
            # Get match details for GUI display
            match_info = {
                'teams': f"{mutated[idx].team1} vs {mutated[idx].team2}",
                'date': mutated[idx].date.strftime('%Y-%m-%d'),
                'time': mutated[idx].time,
                'original_venue': old_venue,
                'new_venue': new_venue
            }
            
            mutation_info['match_details'] = match_info
            
            mutation_info['note'] = f'Venue mutation applied: match {idx} changed from {old_venue} to {new_venue}'
        else:
            mutation_info['note'] = f'No mutation applied (no alternative venues for {old_venue})'
    else:
        mutation_info['note'] = 'No mutation applied (random check failed)'
    
    mutation_info['final_length'] = len(mutated)
    
    return mutated, mutation_info

In [12]:
def change_time_mutation(schedule, match_times, mutation_rate=0.1):
    """
    Change time mutation: change time of a random match
    Returns: (mutated_schedule, mutation_info)
    """
    # Track mutation information for GUI
    mutation_info = {
        'initial_length': len(schedule),
        'available_times_count': len(match_times),
        'mutation_rate': mutation_rate,
        'mutation_applied': False,
        'match_index': None,
        'old_time': None,
        'new_time': None,
        'final_length': 0,
        'operation_type': 'change_time_mutation'
    }
    
    mutated = copy.deepcopy(schedule)
    mutation_info['final_length'] = len(mutated)
    
    if not mutated or len(match_times) < 2:
        mutation_info['note'] = 'No mutation applied (empty schedule or insufficient times)'
        return mutated, mutation_info
    
    # Check if mutation should be applied
    if random.random() < mutation_rate:
        # Select a random match
        idx = random.randint(0, len(mutated) - 1)
        old_time = mutated[idx].time
        
        # Find available alternative times
        available_times = [t for t in match_times if t != old_time]
        
        mutation_info['available_alternative_times'] = len(available_times)
        
        if available_times:
            # Select new time
            new_time = random.choice(available_times)
            
            # Apply the mutation
            mutated[idx].time = new_time
            
            # Record mutation details
            mutation_info['mutation_applied'] = True
            mutation_info['match_index'] = idx
            mutation_info['old_time'] = old_time
            mutation_info['new_time'] = new_time
            
            # Get match details for GUI display
            match_info = {
                'teams': f"{mutated[idx].team1} vs {mutated[idx].team2}",
                'date': mutated[idx].date.strftime('%Y-%m-%d'),
                'original_time': old_time,
                'new_time': new_time,
                'venue': mutated[idx].venue
            }
            
            mutation_info['match_details'] = match_info
            
            mutation_info['note'] = f'Time mutation applied: match {idx} changed from {old_time} to {new_time}'
        else:
            mutation_info['note'] = f'No mutation applied (no alternative times for {old_time})'
    else:
        mutation_info['note'] = 'No mutation applied (random check failed)'
    
    mutation_info['final_length'] = len(mutated)
    
    return mutated, mutation_info

In [13]:
def scramble_mutation(schedule, mutation_rate=0.1):
    """
    Scramble mutation: shuffle a random segment of matches
    Returns: (mutated_schedule, mutation_info)
    """
    # Track mutation information for GUI
    mutation_info = {
        'initial_length': len(schedule),
        'mutation_rate': mutation_rate,
        'mutation_applied': False,
        'segment_start': None,
        'segment_end': None,
        'segment_size': 0,
        'matches_shuffled': 0,
        'final_length': 0,
        'operation_type': 'scramble_mutation'
    }
    
    mutated = copy.deepcopy(schedule)
    mutation_info['final_length'] = len(mutated)
    
    if len(mutated) < 2:
        mutation_info['note'] = 'No mutation applied (schedule too short)'
        return mutated, mutation_info
    
    # Check if mutation should be applied
    if random.random() < mutation_rate:
        # Select random segment boundaries
        start = random.randint(0, len(mutated) - 2)
        end = random.randint(start + 1, len(mutated) - 1)
        segment_size = end - start
        
        # Extract and shuffle the segment
        segment = mutated[start:end]
        
        # Record original order for GUI display
        original_order = []
        for i, match in enumerate(segment):
            original_order.append({
                'original_position': start + i,
                'teams': f"{match.team1} vs {match.team2}",
                'time': match.time,
                'venue': match.venue,
                'date': match.date.strftime('%Y-%m-%d')
            })
        
        # Shuffle the segment
        random.shuffle(segment)
        
        # Apply the shuffled segment back
        mutated[start:end] = segment
        
        # Record mutation details
        mutation_info['mutation_applied'] = True
        mutation_info['segment_start'] = start
        mutation_info['segment_end'] = end
        mutation_info['segment_size'] = segment_size
        mutation_info['matches_shuffled'] = segment_size
        
        # Record new order for comparison
        new_order = []
        for i, match in enumerate(segment):
            new_order.append({
                'new_position': start + i,
                'teams': f"{match.team1} vs {match.team2}",
                'time': match.time,
                'venue': match.venue,
                'date': match.date.strftime('%Y-%m-%d')
            })
        
        # Create detailed comparison for GUI
        comparison_details = []
        for orig, new in zip(original_order, new_order):
            if orig['original_position'] != new['new_position']:
                comparison_details.append({
                    'teams': orig['teams'],
                    'moved_from': orig['original_position'],
                    'moved_to': new['new_position']
                })
        
        mutation_info['original_order'] = original_order
        mutation_info['new_order'] = new_order
        mutation_info['position_changes'] = comparison_details
        
        if comparison_details:
            mutation_info['note'] = f'Scramble mutation applied: {segment_size} matches shuffled (positions {start} to {end})'
        else:
            mutation_info['note'] = f'Scramble mutation applied but no position changes detected'
    else:
        mutation_info['note'] = 'No mutation applied (random check failed)'
    
    mutation_info['final_length'] = len(mutated)
    
    return mutated, mutation_info

In [14]:
def improved_repair_schedule(schedule, teams, venues, all_dates, match_times, min_rest_days=4):
    """
    Improved version of repair_schedule function
    Returns: (repaired_schedule, repair_info)
    """
    # Track repair information for GUI
    repair_info = {
        'initial_schedule_length': len(schedule),
        'repair_attempts_total': 0,
        'matches_repaired': 0,
        'venue_conflicts_resolved': 0,
        'team_conflicts_resolved': 0,
        'rest_periods_fixed': 0,
        'final_schedule_length': 0,
        'repair_details': [],
        'operation_type': 'improved_repair_schedule'
    }
    
    if not schedule:
        repair_info['note'] = 'No repair needed (empty schedule)'
        return [], repair_info
    
    repaired = []
    used_slots = set()  # (date, time, venue)
    team_schedule = {}  # team: [(date, time)]
    team_last_date = {}  # team: last_played_date
    
    # Sort by date to start with oldest
    sorted_schedule = sorted(schedule, key=lambda m: m.date)
    
    for match_idx, match in enumerate(sorted_schedule):
        # Ensure deep copy
        current_match = copy.deepcopy(match)
        
        max_attempts = 20
        placed = False
        attempt_details = {
            'match_index': match_idx,
            'original_teams': f"{match.team1} vs {match.team2}",
            'attempts_needed': 0,
            'changes_made': []
        }
        
        for attempt in range(max_attempts):
            repair_info['repair_attempts_total'] += 1
            attempt_details['attempts_needed'] += 1
            
            changes_in_attempt = []
            
            # 1. Check venue conflict
            venue_conflict = False
            slot_key = (current_match.date, current_match.time, current_match.venue)
            if slot_key in used_slots:
                venue_conflict = True
                # Try different venue
                for new_venue in venues:
                    new_slot = (current_match.date, current_match.time, new_venue)
                    if new_slot not in used_slots:
                        old_venue = current_match.venue
                        current_match.venue = new_venue
                        venue_conflict = False
                        changes_in_attempt.append(f'venue: {old_venue} -> {new_venue}')
                        repair_info['venue_conflicts_resolved'] += 1
                        break
            
            # 2. Check team time conflict
            team_conflict = False
            for team in [current_match.team1, current_match.team2]:
                if team in team_schedule:
                    if (current_match.date, current_match.time) in team_schedule[team]:
                        team_conflict = True
                        break
            
            if team_conflict:
                # Try different time
                for new_time in match_times:
                    # Check venue with new time
                    new_slot = (current_match.date, new_time, current_match.venue)
                    if new_slot not in used_slots:
                        # Check team conflict with new time
                        new_time_ok = True
                        for team in [current_match.team1, current_match.team2]:
                            if team in team_schedule:
                                if (current_match.date, new_time) in team_schedule[team]:
                                    new_time_ok = False
                                    break
                        
                        if new_time_ok:
                            old_time = current_match.time
                            current_match.time = new_time
                            team_conflict = False
                            changes_in_attempt.append(f'time: {old_time} -> {new_time}')
                            repair_info['team_conflicts_resolved'] += 1
                            break
            
            # 3. Check rest period
            rest_violation = False
            for team in [current_match.team1, current_match.team2]:
                if team in team_last_date:
                    days_diff = (current_match.date - team_last_date[team]).days
                    if days_diff < min_rest_days:
                        rest_violation = True
                        break
            
            if rest_violation:
                # Try new date
                possible_dates = []
                for date in all_dates:
                    date_ok = True
                    # Check conflicts with new date
                    for team in [current_match.team1, current_match.team2]:
                        if team in team_schedule:
                            if (date, current_match.time) in team_schedule[team]:
                                date_ok = False
                                break
                    
                    if date_ok:
                        possible_dates.append(date)
                
                if possible_dates:
                    old_date = current_match.date
                    current_match.date = possible_dates[0]
                    rest_violation = False
                    changes_in_attempt.append(f'date: {old_date.strftime("%Y-%m-%d")} -> {current_match.date.strftime("%Y-%m-%d")}')
                    repair_info['rest_periods_fixed'] += 1
            
            # 4. If everything is OK
            if not venue_conflict and not team_conflict and not rest_violation:
                placed = True
                break
            
            # 5. If failed, regenerate everything
            current_match.date = random.choice(all_dates)
            current_match.time = random.choice(match_times)
            current_match.venue = random.choice(venues)
            changes_in_attempt.append('complete_regeneration')
        
        if changes_in_attempt:
            attempt_details['changes_made'] = changes_in_attempt
        
        if placed:
            repaired.append(current_match)
            repair_info['matches_repaired'] += 1
            repair_info['repair_details'].append(attempt_details)
            
            # Update records
            final_slot = (current_match.date, current_match.time, current_match.venue)
            used_slots.add(final_slot)
            
            for team in [current_match.team1, current_match.team2]:
                if team not in team_schedule:
                    team_schedule[team] = []
                team_schedule[team].append((current_match.date, current_match.time))
                team_last_date[team] = current_match.date
        else:
            # Could not place the match even after max attempts
            repair_info['repair_details'].append({
                'match_index': match_idx,
                'original_teams': f"{match.team1} vs {match.team2}",
                'status': 'failed',
                'note': 'Could not place after maximum attempts'
            })
    
    repair_info['final_schedule_length'] = len(repaired)
    repair_info['success_rate'] = round((repair_info['matches_repaired'] / repair_info['initial_schedule_length']) * 100, 1) if repair_info['initial_schedule_length'] > 0 else 100
    
    if repair_info['matches_repaired'] == repair_info['initial_schedule_length']:
        repair_info['note'] = f'All {repair_info["matches_repaired"]} matches successfully repaired'
    else:
        repair_info['note'] = f'{repair_info["matches_repaired"]} of {repair_info["initial_schedule_length"]} matches repaired'
    
    return repaired, repair_info

In [15]:
def tournament_selection(population, fitness_scores, tournament_size=3):
    """
    Tournament selection: select parent using tournament method
    Returns: (selected_schedule, selection_info)
    """
    # Track selection information for GUI
    selection_info = {
        'population_size': len(population),
        'tournament_size': min(tournament_size, len(population)),
        'candidates_evaluated': 0,
        'winner_index': None,
        'winner_fitness': None,
        'candidate_fitness_scores': [],
        'operation_type': 'tournament_selection'
    }
    
    if len(population) < tournament_size:
        tournament_size = len(population)
        selection_info['adjusted_tournament_size'] = tournament_size
    
    # Select random candidates for tournament
    tournament_indices = random.sample(range(len(population)), tournament_size)
    selection_info['candidate_indices'] = tournament_indices
    
    # Start with first candidate as winner
    winner_idx = tournament_indices[0]
    winner_fitness = fitness_scores[winner_idx]
    
    # Record first candidate details
    selection_info['candidate_fitness_scores'].append({
        'candidate_index': tournament_indices[0],
        'fitness_score': fitness_scores[tournament_indices[0]],
        'status': 'initial_winner'
    })
    
    # Tournament competition
    for i, idx in enumerate(tournament_indices[1:], start=1):
        selection_info['candidates_evaluated'] += 1
        
        # Record candidate details
        candidate_info = {
            'candidate_index': idx,
            'fitness_score': fitness_scores[idx],
            'compared_to_winner': winner_fitness
        }
        
        # Compare fitness
        if fitness_scores[idx] > winner_fitness:
            candidate_info['status'] = 'new_winner'
            winner_idx = idx
            winner_fitness = fitness_scores[idx]
        else:
            candidate_info['status'] = 'lost'
        
        selection_info['candidate_fitness_scores'].append(candidate_info)
    
    # Final results
    selection_info['winner_index'] = winner_idx
    selection_info['winner_fitness'] = winner_fitness
    
    # Calculate statistics
    all_fitness_in_tournament = [fitness_scores[i] for i in tournament_indices]
    selection_info['tournament_average_fitness'] = round(sum(all_fitness_in_tournament) / len(all_fitness_in_tournament), 2)
    selection_info['tournament_max_fitness'] = max(all_fitness_in_tournament)
    selection_info['tournament_min_fitness'] = min(all_fitness_in_tournament)
    
    # Check if winner is the best in tournament
    if winner_fitness == selection_info['tournament_max_fitness']:
        selection_info['winner_is_best_in_tournament'] = True
        selection_info['note'] = f'Winner selected (index {winner_idx}) with fitness {winner_fitness:.2f}'
    else:
        selection_info['winner_is_best_in_tournament'] = False
        selection_info['note'] = f'Winner selected (index {winner_idx}) with fitness {winner_fitness:.2f}, but best was {selection_info["tournament_max_fitness"]:.2f}'
    
    return population[winner_idx], selection_info

In [16]:
def apply_crossover(parent1, parent2, crossover_type="single_point"):
    """
    Apply crossover with automatic repair
    Returns: (child_schedule, crossover_application_info)
    """
    # Track application information for GUI
    application_info = {
        'parent1_length': len(parent1),
        'parent2_length': len(parent2),
        'selected_crossover_type': crossover_type,
        'crossover_successful': False,
        'repair_applied': False,
        'final_child_length': 0,
        'operation_type': 'apply_crossover',
        'error_message': None
    }
    
    crossover_funcs = {
        "single_point": single_point_crossover,
        "two_point": two_point_crossover,
        "uniform": uniform_crossover
    }
    
    if crossover_type not in crossover_funcs:
        error_msg = f"Unknown crossover type: {crossover_type}"
        application_info['error_message'] = error_msg
        application_info['note'] = f'Error: {error_msg}'
        raise ValueError(error_msg)
    
    try:
        # Apply selected crossover
        crossover_function = crossover_funcs[crossover_type]
        child_schedule, crossover_details = crossover_function(parent1, parent2)
        
        application_info['crossover_successful'] = True
        application_info['crossover_details'] = crossover_details
        application_info['initial_child_length'] = len(child_schedule)
        
        # Automatic repair if needed
        if child_schedule and len(child_schedule) > 0:
            # Use global variables (assuming they're imported/available)
            # Check if global variables exist
            if 'teams' in globals() and 'venues' in globals() and 'all_dates' in globals() and 'match_times' in globals():
                # Try to import if not available
                try:
                    # Apply repair
                    repaired_schedule, repair_details = improved_repair_schedule(
                        child_schedule, 
                        teams, 
                        venues, 
                        all_dates, 
                        match_times
                    )
                    
                    # Update child with repaired version
                    child_schedule = repaired_schedule
                    application_info['repair_applied'] = True
                    application_info['repair_details'] = repair_details
                    application_info['repair_improvement'] = len(repaired_schedule) - len(child_schedule)
                    
                    if len(repaired_schedule) > len(child_schedule):
                        application_info['repair_note'] = f'Repair added {application_info["repair_improvement"]} matches'
                    elif len(repaired_schedule) < len(child_schedule):
                        application_info['repair_note'] = f'Repair removed {abs(application_info["repair_improvement"])} matches'
                    else:
                        application_info['repair_note'] = 'Repair did not change schedule length'
                except Exception as repair_error:
                    application_info['repair_error'] = str(repair_error)
                    application_info['repair_note'] = f'Repair failed: {repair_error}'
            else:
                application_info['repair_note'] = 'Repair skipped (global variables not available)'
                application_info['repair_skipped_reason'] = 'Missing global variables'
        
        # Final child information
        application_info['final_child_length'] = len(child_schedule)
        
        # Success message
        application_info['note'] = f'{crossover_type.replace("_", " ").title()} applied successfully'
        
        return child_schedule, application_info
        
    except Exception as e:
        error_msg = str(e)
        application_info['error_message'] = error_msg
        application_info['note'] = f'Error during crossover: {error_msg}'
        application_info['crossover_successful'] = False
        
        # Return empty schedule and error info
        return [], application_info

In [17]:
def apply_mutation(schedule, venues=None, match_times=None, mutation_types=None):
    """
    Apply multiple types of mutations to the schedule
    Returns: (mutated_schedule, mutation_application_info)
    """
    # Track application information for GUI
    application_info = {
        'initial_schedule_length': len(schedule),
        'mutation_types_requested': mutation_types if mutation_types else ["swap", "change_venue", "change_time", "scramble"],
        'mutation_types_applied': [],
        'venues_provided': venues is not None,
        'match_times_provided': match_times is not None,
        'mutations_applied_count': 0,
        'final_schedule_length': 0,
        'mutation_results': [],
        'operation_type': 'apply_mutation'
    }
    
    if mutation_types is None:
        mutation_types = ["swap", "change_venue", "change_time", "scramble"]
    
    mutated = copy.deepcopy(schedule)
    
    # Track if schedule was actually modified
    schedule_modified = False
    
    for mutation_type in mutation_types:
        mutation_result = {
            'mutation_type': mutation_type,
            'applied': False,
            'details': None,
            'error': None
        }
        
        try:
            if mutation_type == "swap":
                mutated, swap_info = swap_mutation(mutated)
                mutation_result['applied'] = swap_info['mutation_applied']
                mutation_result['details'] = swap_info
                if swap_info['mutation_applied']:
                    application_info['mutation_types_applied'].append(mutation_type)
                    application_info['mutations_applied_count'] += 1
                    schedule_modified = True
                    
            elif mutation_type == "change_venue" and venues:
                mutated, venue_info = change_venue_mutation(mutated, venues)
                mutation_result['applied'] = venue_info['mutation_applied']
                mutation_result['details'] = venue_info
                if venue_info['mutation_applied']:
                    application_info['mutation_types_applied'].append(mutation_type)
                    application_info['mutations_applied_count'] += 1
                    schedule_modified = True
                    
            elif mutation_type == "change_time" and match_times:
                mutated, time_info = change_time_mutation(mutated, match_times)
                mutation_result['applied'] = time_info['mutation_applied']
                mutation_result['details'] = time_info
                if time_info['mutation_applied']:
                    application_info['mutation_types_applied'].append(mutation_type)
                    application_info['mutations_applied_count'] += 1
                    schedule_modified = True
                    
            elif mutation_type == "scramble":
                mutated, scramble_info = scramble_mutation(mutated)
                mutation_result['applied'] = scramble_info['mutation_applied']
                mutation_result['details'] = scramble_info
                if scramble_info['mutation_applied']:
                    application_info['mutation_types_applied'].append(mutation_type)
                    application_info['mutations_applied_count'] += 1
                    schedule_modified = True
                    
            else:
                if mutation_type in ["change_venue", "change_time"]:
                    missing_resource = "venues" if mutation_type == "change_venue" else "match_times"
                    mutation_result['error'] = f'{mutation_type} mutation skipped: {missing_resource} not provided'
                else:
                    mutation_result['error'] = f'Unknown mutation type: {mutation_type}'
                    
        except Exception as e:
            mutation_result['error'] = str(e)
            mutation_result['applied'] = False
        
        application_info['mutation_results'].append(mutation_result)
    
    # Final schedule information
    application_info['final_schedule_length'] = len(mutated)
    
    # Summary statistics
    successful_mutations = sum(1 for result in application_info['mutation_results'] if result['applied'])
    application_info['successful_mutations_count'] = successful_mutations
    
    if successful_mutations > 0:
        application_info['note'] = f'Applied {successful_mutations} mutation(s) successfully'
    else:
        application_info['note'] = 'No mutations were applied (mutation rate or conditions prevented application)'
    
    application_info['schedule_modified'] = schedule_modified
    
    # If no mutations were applied, note that original schedule is returned
    if not schedule_modified:
        application_info['note'] += ' - Original schedule returned unchanged'
    
    return mutated, application_info

In [18]:
# ==================== Cell: Check Required Functions ====================

print("Checking basic functions...")

required_functions = [
    'single_point_crossover',
    'two_point_crossover', 
    'uniform_crossover',
    'swap_mutation',
    'change_venue_mutation',
    'change_time_mutation',
    'scramble_mutation',
    'apply_crossover',
    'apply_mutation',
    'tournament_selection'
]

missing_functions = []
for func in required_functions:
    if func in globals():
        print(f"   Found: {func}")
    else:
        print(f"   Missing: {func}")
        missing_functions.append(func)

if missing_functions:
    print(f"Error: Missing functions: {missing_functions}")
else:
    print("All required functions are available")

Checking basic functions...
   Found: single_point_crossover
   Found: two_point_crossover
   Found: uniform_crossover
   Found: swap_mutation
   Found: change_venue_mutation
   Found: change_time_mutation
   Found: scramble_mutation
   Found: apply_crossover
   Found: apply_mutation
   Found: tournament_selection
All required functions are available


In [19]:
# ==================== Cell: Test Crossover Operations ====================

print("\nTesting Crossover Operations...")

# Generate test schedules
parent1 = generate_weekly_schedule(teams[:3], venues[:2], all_dates[:5], match_times)
parent2 = generate_weekly_schedule(teams[:3], venues[:2], all_dates[:5], match_times)

print(f"   Parent 1: {len(parent1)} matches")
print(f"   Parent 2: {len(parent2)} matches")

# Test Single Point Crossover
child1 = apply_crossover(parent1, parent2, "single_point")
print(f"   Single Point Crossover: {len(child1)} matches")

# Test Two Point Crossover
child2 = apply_crossover(parent1, parent2, "two_point")
print(f"   Two Point Crossover: {len(child2)} matches")

# Test Uniform Crossover
child3 = apply_crossover(parent1, parent2, "uniform")
print(f"   Uniform Crossover: {len(child3)} matches")

print("All crossover operations completed")


Testing Crossover Operations...
   Parent 1: 1 matches
   Parent 2: 1 matches
   Single Point Crossover: 2 matches
   Two Point Crossover: 2 matches
   Uniform Crossover: 2 matches
All crossover operations completed


In [20]:
# ==================== Cell: Test Mutation Operations ====================

print("\nTesting Mutation Operations...")

# Generate test schedule
test_schedule = generate_weekly_schedule(teams[:3], venues[:2], all_dates[:5], match_times)
print(f"   Test schedule: {len(test_schedule)} matches")

# Test Swap Mutation
mutated1 = apply_mutation(test_schedule, venues, match_times, ["swap"])
print(f"   Swap Mutation applied")

# Test Change Venue Mutation
mutated2 = apply_mutation(test_schedule, venues, match_times, ["change_venue"])
print(f"   Change Venue Mutation applied")

# Test Change Time Mutation
mutated3 = apply_mutation(test_schedule, venues, match_times, ["change_time"])
print(f"   Change Time Mutation applied")

# Test Scramble Mutation
mutated4 = apply_mutation(test_schedule, venues, match_times, ["scramble"])
print(f"   Scramble Mutation applied")

# Test All Mutations
mutated_all = apply_mutation(test_schedule, venues, match_times)
print(f"   All Mutations applied")

print("All mutation operations completed")


Testing Mutation Operations...
   Test schedule: 1 matches
   Swap Mutation applied
   Change Venue Mutation applied
   Change Time Mutation applied
   Scramble Mutation applied
   All Mutations applied
All mutation operations completed


In [21]:
# ==================== Cell: Test Selection Operation ====================

print("\nTesting Selection Operation...")

# Create test population
population = []
fitness_scores = []

for i in range(5):
    schedule = generate_weekly_schedule(teams[:3], venues[:2], all_dates[:5], match_times)
    population.append(schedule)
    fitness_scores.append(random.uniform(70, 90))  # Example fitness scores

print(f"   Population created: {len(population)} schedules")

# Test Tournament Selection
selected_schedule, selection_info = tournament_selection(population, fitness_scores)
selected_index = selection_info['winner_index']

print(f"   Tournament Selection completed")
print(f"   Selected schedule index: {selected_index}")
print(f"   Selected schedule fitness: {selection_info['winner_fitness']:.2f}")

# Additional details for GUI
print(f"   Tournament candidates: {selection_info['candidate_indices']}")
print(f"   Tournament average fitness: {selection_info['tournament_average_fitness']:.2f}")
print(f"   Best in tournament: {selection_info['winner_is_best_in_tournament']}")

print("Selection operation completed")


Testing Selection Operation...
   Population created: 5 schedules
   Tournament Selection completed
   Selected schedule index: 4
   Selected schedule fitness: 89.64
   Tournament candidates: [2, 1, 4]
   Tournament average fitness: 81.22
   Best in tournament: True
Selection operation completed


In [22]:
# ==================== Cell: Final Summary ====================

print("\n" + "=" * 50)
print("Task 3 Test Summary")
print("=" * 50)
print("Genetic Algorithm Operations Testing Complete")
print("=" * 50)
print("\nResults:")
print("-" * 20)
print("Crossover Operations: 3 types tested")
print("Mutation Operations: 4 types tested")
print("Selection Operation: Tournament selection tested")
print("Integration: Works with schedule generation and fitness")
print("\nStatus: Task 3 implementation is complete and functional")
print("=" * 50)


Task 3 Test Summary
Genetic Algorithm Operations Testing Complete

Results:
--------------------
Crossover Operations: 3 types tested
Mutation Operations: 4 types tested
Selection Operation: Tournament selection tested
Integration: Works with schedule generation and fitness

Status: Task 3 implementation is complete and functional


In [23]:
# # ==================== Cell: GUI for Genetic Algorithm Operations ====================

# import ipywidgets as widgets
# from IPython.display import display, clear_output
# import pandas as pd
# import random
# import copy
# from datetime import datetime

# # تعريف البيانات اللازمة للاختبار
# # يمكن تعديل هذه البيانات حسب احتياجاتك

# teams = [f"Team {i}" for i in range(1, 9)]
# venues = [f"Venue {i}" for i in range(1, 6)]
# all_dates = [datetime(2024, 1, i) for i in range(1, 16)]
# match_times = ["14:00", "16:30", "19:00", "21:30"]

# # فئة المباراة
# class Match:
#     def __init__(self, team1, team2, venue, date, time, match_type="league"):
#         self.team1 = team1
#         self.team2 = team2
#         self.venue = venue
#         self.date = date
#         self.time = time
#         self.match_type = match_type
    
#     def __repr__(self):
#         return f"{self.team1} vs {self.team2} at {self.venue} on {self.date.strftime('%Y-%m-%d')} {self.time}"

# # دالة لإنشاء جدول اختباري
# def create_test_schedule(num_matches=10):
#     schedule = []
#     for i in range(num_matches):
#         team1 = random.choice(teams)
#         team2 = random.choice([t for t in teams if t != team1])
#         venue = random.choice(venues)
#         date = random.choice(all_dates)
#         time = random.choice(match_times)
        
#         schedule.append(Match(team1, team2, venue, date, time))
#     return schedule

# # إنشاء بيانات اختبارية
# parent1 = create_test_schedule(8)
# parent2 = create_test_schedule(8)
# current_schedule = create_test_schedule(12)
# population = [create_test_schedule(random.randint(5, 10)) for _ in range(10)]
# fitness_scores = [random.uniform(0.5, 0.95) for _ in range(10)]

# # ==================== واجهة المستخدم ====================

# # عناصر التحكم
# operation_dropdown = widgets.Dropdown(
#     options=[
#         ('Crossover', 'crossover'),
#         ('Swap Mutation', 'swap_mutation'),
#         ('Change Venue Mutation', 'change_venue_mutation'),
#         ('Change Time Mutation', 'change_time_mutation'),
#         ('Scramble Mutation', 'scramble_mutation'),
#         ('Repair Schedule', 'repair'),
#         ('Tournament Selection', 'tournament_selection')
#     ],
#     value='crossover',
#     description='Operation:',
#     style={'description_width': 'initial'}
# )

# crossover_type_dropdown = widgets.Dropdown(
#     options=[
#         ('Single Point', 'single_point'),
#         ('Two Point', 'two_point'),
#         ('Uniform', 'uniform')
#     ],
#     value='single_point',
#     description='Crossover Type:',
#     style={'description_width': 'initial'},
#     layout={'width': '300px'}
# )

# mutation_type_dropdown = widgets.Dropdown(
#     options=[
#         ('Swap', 'swap'),
#         ('Change Venue', 'change_venue'),
#         ('Change Time', 'change_time'),
#         ('Scramble', 'scramble'),
#         ('All Mutations', 'all')
#     ],
#     value='swap',
#     description='Mutation Type:',
#     style={'description_width': 'initial'},
#     layout={'width': '300px'}
# )

# mutation_rate_slider = widgets.FloatSlider(
#     value=0.3,
#     min=0.0,
#     max=1.0,
#     step=0.05,
#     description='Mutation Rate:',
#     style={'description_width': 'initial'}
# )

# tournament_size_slider = widgets.IntSlider(
#     value=3,
#     min=2,
#     max=10,
#     step=1,
#     description='Tournament Size:',
#     style={'description_width': 'initial'}
# )

# detailed_output_checkbox = widgets.Checkbox(
#     value=True,
#     description='Show detailed info',
#     style={'description_width': 'initial'}
# )

# show_schedule_checkbox = widgets.Checkbox(
#     value=False,
#     description='Show schedule details',
#     style={'description_width': 'initial'}
# )

# run_button = widgets.Button(
#     description='Run Operation',
#     button_style='primary',
#     icon='play',
#     layout={'width': '200px'}
# )

# reset_button = widgets.Button(
#     description='Reset Test Data',
#     button_style='warning',
#     icon='refresh',
#     layout={'width': '200px'}
# )

# output = widgets.Output()

# # دالة لتحديث عناصر التحكم بناءً على العملية المختارة
# def update_controls(change):
#     with output:
#         clear_output(wait=True)
    
#     op_type = change['new']
    
#     # إظهار/إخفاء عناصر التحكم حسب العملية
#     if op_type == 'crossover':
#         crossover_type_dropdown.layout.visibility = 'visible'
#         mutation_type_dropdown.layout.visibility = 'hidden'
#         mutation_rate_slider.layout.visibility = 'hidden'
#         tournament_size_slider.layout.visibility = 'hidden'
#     elif op_type in ['swap_mutation', 'change_venue_mutation', 'change_time_mutation', 'scramble_mutation']:
#         crossover_type_dropdown.layout.visibility = 'hidden'
#         mutation_type_dropdown.layout.visibility = 'visible'
#         mutation_rate_slider.layout.visibility = 'visible'
#         tournament_size_slider.layout.visibility = 'hidden'
#     elif op_type == 'tournament_selection':
#         crossover_type_dropdown.layout.visibility = 'hidden'
#         mutation_type_dropdown.layout.visibility = 'hidden'
#         mutation_rate_slider.layout.visibility = 'hidden'
#         tournament_size_slider.layout.visibility = 'visible'
#     else:
#         crossover_type_dropdown.layout.visibility = 'hidden'
#         mutation_type_dropdown.layout.visibility = 'hidden'
#         mutation_rate_slider.layout.visibility = 'hidden'
#         tournament_size_slider.layout.visibility = 'hidden'

# # ربط دالة التحديث
# operation_dropdown.observe(update_controls, names='value')

# # تهيئة عناصر التحكم
# update_controls({'new': operation_dropdown.value})

# # دالة لإعادة تعيين البيانات الاختبارية
# def reset_test_data(b):
#     with output:
#         clear_output()
#         print("Resetting test data...")
        
#         global parent1, parent2, current_schedule, population, fitness_scores
        
#         parent1 = create_test_schedule(8)
#         parent2 = create_test_schedule(8)
#         current_schedule = create_test_schedule(12)
#         population = [create_test_schedule(random.randint(5, 10)) for _ in range(10)]
#         fitness_scores = [random.uniform(0.5, 0.95) for _ in range(10)]
        
#         print("✓ Test data reset successfully!")
#         print(f"  - Parent 1: {len(parent1)} matches")
#         print(f"  - Parent 2: {len(parent2)} matches")
#         print(f"  - Current schedule: {len(current_schedule)} matches")
#         print(f"  - Population: {len(population)} schedules")
#         print(f"  - Fitness scores range: {min(fitness_scores):.2f} - {max(fitness_scores):.2f}")

# # دالة لعرض معلومات الجدول
# def display_schedule_info(schedule, title="Schedule"):
#     print(f"\n{title} Information:")
#     print("-" * 40)
#     print(f"Total matches: {len(schedule)}")
    
#     if len(schedule) > 0:
#         # تحليل البيانات
#         venues_used = {}
#         dates_used = {}
#         teams_used = {}
        
#         for match in schedule[:10]:  # عرض أول 10 مباريات فقط
#             venues_used[match.venue] = venues_used.get(match.venue, 0) + 1
#             date_str = match.date.strftime('%Y-%m-%d')
#             dates_used[date_str] = dates_used.get(date_str, 0) + 1
#             teams_used[match.team1] = teams_used.get(match.team1, 0) + 1
#             teams_used[match.team2] = teams_used.get(match.team2, 0) + 1
        
#         print(f"Venues used: {len(venues_used)}")
#         print(f"Dates used: {len(dates_used)}")
#         print(f"Teams involved: {len(teams_used)}")
        
#         # عرض أول 3 مباريات كمثال
#         print("\nSample matches:")
#         for i, match in enumerate(schedule[:3], 1):
#             print(f"  {i}. {match.team1} vs {match.team2} | {match.venue} | {match.date.strftime('%Y-%m-%d')} {match.time}")

# # دالة تنفيذ العمليات
# def run_operation(b):
#     with output:
#         clear_output()
#         op_type = operation_dropdown.value
#         show_details = detailed_output_checkbox.value
#         show_schedule = show_schedule_checkbox.value
        
#         try:
#             if op_type == 'crossover':
#                 print("=" * 60)
#                 print("CROSSOVER OPERATION")
#                 print("=" * 60)
                
#                 if show_schedule:
#                     display_schedule_info(parent1, "Parent 1")
#                     display_schedule_info(parent2, "Parent 2")
                
#                 child, info = apply_crossover(parent1, parent2, crossover_type_dropdown.value)
                
#                 print(f"\nCrossover Type: {crossover_type_dropdown.value.replace('_', ' ').title()}")
#                 print(f"Result: Child with {len(child)} matches")
                
#                 if show_schedule:
#                     display_schedule_info(child, "Child Schedule")
                
#                 if show_details:
#                     print("\nDetailed Information:")
#                     print("-" * 40)
#                     for key, value in info.items():
#                         if key not in ['crossover_details', 'repair_details']:
#                             print(f"{key}: {value}")
                    
#                     if 'crossover_details' in info:
#                         print("\nCrossover Details:")
#                         for k, v in info['crossover_details'].items():
#                             if k not in ['original_order', 'new_order', 'match_details']:
#                                 print(f"  {k}: {v}")
                
#                 # تحديث الجدول الحالي بنتيجة التزاوج
#                 global current_schedule
#                 current_schedule = child
                
#             elif op_type == 'swap_mutation':
#                 print("=" * 60)
#                 print("SWAP MUTATION")
#                 print("=" * 60)
                
#                 if show_schedule:
#                     display_schedule_info(current_schedule, "Original Schedule")
                
#                 # تحديث معدل التحور
#                 mutated, info = swap_mutation(current_schedule, mutation_rate_slider.value)
                
#                 print(f"\nMutation Rate: {mutation_rate_slider.value}")
#                 print(f"Mutation Applied: {info.get('mutation_applied', False)}")
                
#                 if show_schedule:
#                     display_schedule_info(mutated, "Mutated Schedule")
                
#                 if show_details:
#                     print("\nDetailed Information:")
#                     print("-" * 40)
#                     for key, value in info.items():
#                         if key not in ['match_details', 'original_order', 'new_order']:
#                             print(f"{key}: {value}")
                    
#                     if 'match_details' in info:
#                         print("\nSwap Details:")
#                         details = info['match_details']
#                         if isinstance(details, dict) and 'match1' in details:
#                             print(f"  Match 1 moved from position {details['match1'].get('original_position')} to {details['match1'].get('new_position')}")
#                             print(f"  Match 2 moved from position {details['match2'].get('original_position')} to {details['match2'].get('new_position')}")
                
#                 # تحديث الجدول الحالي
#                 current_schedule = mutated
                
#             elif op_type == 'change_venue_mutation':
#                 print("=" * 60)
#                 print("CHANGE VENUE MUTATION")
#                 print("=" * 60)
                
#                 if show_schedule:
#                     display_schedule_info(current_schedule, "Original Schedule")
                
#                 mutated, info = change_venue_mutation(current_schedule, venues, mutation_rate_slider.value)
                
#                 print(f"\nMutation Rate: {mutation_rate_slider.value}")
#                 print(f"Mutation Applied: {info.get('mutation_applied', False)}")
                
#                 if info.get('mutation_applied'):
#                     print(f"  Changed venue of match {info.get('match_index')}")
#                     print(f"  From: {info.get('old_venue')}")
#                     print(f"  To: {info.get('new_venue')}")
                
#                 if show_schedule:
#                     display_schedule_info(mutated, "Mutated Schedule")
                
#                 if show_details:
#                     print("\nDetailed Information:")
#                     print("-" * 40)
#                     for key, value in info.items():
#                         if key != 'match_details':
#                             print(f"{key}: {value}")
                
#                 current_schedule = mutated
                
#             elif op_type == 'change_time_mutation':
#                 print("=" * 60)
#                 print("CHANGE TIME MUTATION")
#                 print("=" * 60)
                
#                 if show_schedule:
#                     display_schedule_info(current_schedule, "Original Schedule")
                
#                 mutated, info = change_time_mutation(current_schedule, match_times, mutation_rate_slider.value)
                
#                 print(f"\nMutation Rate: {mutation_rate_slider.value}")
#                 print(f"Mutation Applied: {info.get('mutation_applied', False)}")
                
#                 if info.get('mutation_applied'):
#                     print(f"  Changed time of match {info.get('match_index')}")
#                     print(f"  From: {info.get('old_time')}")
#                     print(f"  To: {info.get('new_time')}")
                
#                 if show_schedule:
#                     display_schedule_info(mutated, "Mutated Schedule")
                
#                 if show_details:
#                     print("\nDetailed Information:")
#                     print("-" * 40)
#                     for key, value in info.items():
#                         if key != 'match_details':
#                             print(f"{key}: {value}")
                
#                 current_schedule = mutated
                
#             elif op_type == 'scramble_mutation':
#                 print("=" * 60)
#                 print("SCRAMBLE MUTATION")
#                 print("=" * 60)
                
#                 if show_schedule:
#                     display_schedule_info(current_schedule, "Original Schedule")
                
#                 mutated, info = scramble_mutation(current_schedule, mutation_rate_slider.value)
                
#                 print(f"\nMutation Rate: {mutation_rate_slider.value}")
#                 print(f"Mutation Applied: {info.get('mutation_applied', False)}")
                
#                 if info.get('mutation_applied'):
#                     print(f"  Shuffled segment from position {info.get('segment_start')} to {info.get('segment_end')}")
#                     print(f"  Matches shuffled: {info.get('matches_shuffled')}")
                
#                 if show_schedule:
#                     display_schedule_info(mutated, "Mutated Schedule")
                
#                 if show_details:
#                     print("\nDetailed Information:")
#                     print("-" * 40)
#                     for key, value in info.items():
#                         if key not in ['original_order', 'new_order', 'position_changes']:
#                             print(f"{key}: {value}")
                
#                 current_schedule = mutated
                
#             elif op_type == 'repair':
#                 print("=" * 60)
#                 print("REPAIR SCHEDULE")
#                 print("=" * 60)
                
#                 if show_schedule:
#                     display_schedule_info(current_schedule, "Original Schedule")
                
#                 repaired, info = improved_repair_schedule(current_schedule, teams, venues, all_dates, match_times)
                
#                 print(f"\nRepair Results:")
#                 print(f"  Initial matches: {info.get('initial_schedule_length')}")
#                 print(f"  Repaired matches: {info.get('final_schedule_length')}")
#                 print(f"  Success rate: {info.get('success_rate', 0)}%")
                
#                 if show_schedule:
#                     display_schedule_info(repaired, "Repaired Schedule")
                
#                 if show_details:
#                     print("\nDetailed Information:")
#                     print("-" * 40)
#                     for key, value in info.items():
#                         if key not in ['repair_details']:
#                             print(f"{key}: {value}")
                
#                 current_schedule = repaired
                
#             elif op_type == 'tournament_selection':
#                 print("=" * 60)
#                 print("TOURNAMENT SELECTION")
#                 print("=" * 60)
                
#                 selected, info = tournament_selection(population, fitness_scores, tournament_size_slider.value)
                
#                 print(f"\nTournament Size: {tournament_size_slider.value}")
#                 print(f"Population Size: {len(population)}")
#                 print(f"\nSelection Results:")
#                 print(f"  Winner index: {info.get('winner_index')}")
#                 print(f"  Winner fitness: {info.get('winner_fitness', 0):.3f}")
#                 print(f"  Tournament average fitness: {info.get('tournament_average_fitness', 0):.3f}")
#                 print(f"  Best in tournament: {info.get('winner_is_best_in_tournament', False)}")
                
#                 if show_schedule:
#                     display_schedule_info(selected, "Selected Schedule")
                
#                 if show_details:
#                     print("\nDetailed Information:")
#                     print("-" * 40)
#                     for key, value in info.items():
#                         if key not in ['candidate_fitness_scores']:
#                             print(f"{key}: {value}")
                    
#                     if 'candidate_fitness_scores' in info:
#                         print("\nCandidate Fitness Scores:")
#                         for candidate in info['candidate_fitness_scores']:
#                             print(f"  Candidate {candidate.get('candidate_index')}: {candidate.get('fitness_score', 0):.3f} ({candidate.get('status', '')})")
            
#             print("\n" + "=" * 60)
#             print("Operation completed successfully!")
#             print("=" * 60)
            
#         except Exception as e:
#             print(f"\n❌ Error: {e}")
#             import traceback
#             traceback.print_exc()

# # ربط الأزرار بالدوال
# run_button.on_click(run_operation)
# reset_button.on_click(reset_test_data)

# # عرض واجهة المستخدم
# print("Genetic Algorithm Operations GUI")
# print("=" * 50)
# print("Select an operation and click 'Run Operation' to test.")

# # ترتيب عناصر الواجهة
# controls_box = widgets.VBox([
#     operation_dropdown,
#     crossover_type_dropdown,
#     mutation_type_dropdown,
#     mutation_rate_slider,
#     tournament_size_slider,
#     widgets.HBox([detailed_output_checkbox, show_schedule_checkbox]),
#     widgets.HBox([run_button, reset_button])
# ])

# display(controls_box)
# display(output)

# # عرض معلومات البيانات الحالية
# with output:
#     print("\nCurrent Test Data Status:")
#     print("-" * 30)
#     print(f"• Teams: {len(teams)} teams")
#     print(f"• Venues: {len(venues)} venues")
#     print(f"• Dates: {len(all_dates)} dates")
#     print(f"• Match Times: {len(match_times)} time slots")
#     print(f"• Current Schedule: {len(current_schedule)} matches")
#     print(f"• Population: {len(population)} schedules")
#     print(f"• Fitness Scores: {len(fitness_scores)} values")

In [24]:
# ==================== Cell: Comprehensive GUI for Genetic Algorithm Operations ====================

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import random
import copy
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# تأكد من وجود جميع الدوال السابقة
print("Loading all genetic algorithm operations...")

# تعريف الـ Match كما في الكود السابق
class Match:
    def __init__(self, team1, team2, date, time, venue):
        self.team1 = team1
        self.team2 = team2
        self.date = date
        self.time = time
        self.venue = venue

    def __repr__(self):
        return f"{self.team1} vs {self.team2} on {self.date.strftime('%Y-%m-%d')} at {self.time} in {self.venue}"

# بيانات الفرق والملاعب والأوقات (نفس البيانات السابقة)
teams = [
    "Al Ahly", "Zamalek", "Pyramids", "Masry", "Future", "Ismaily",
    "Smouha", "ENPPI", "Ceramica", "National Bank", "Talaea El Gaish",
    "Alexandria Union", "El Dakhleya", "El Gouna", "Zed",
    "Modern Sport", "Pharco", "Wadi Degla"
]

venues = [
    "Cairo Stadium", "Borg El Arab", "Air Defense Stadium",
    "Suez Stadium", "Alexandria Stadium", "Petro Sport Stadium",
    "Military Academy Stadium", "Al Salam Stadium", "El Sekka El Hadeed Stadium",
    "Zed Club Stadium"
]

match_times = ["17:00", "20:00"]

# مدة البطولة
start_date = datetime(2025, 5, 1)
end_date = datetime(2026, 1, 31)

# إعداد أيام البطولة
all_dates = []
current_date = start_date
while current_date <= end_date:
    all_dates.append(current_date)
    current_date += timedelta(days=1)

# توليد جدول دوري أسبوعي مع تكرار المباريات مرتين
def generate_weekly_schedule(teams, venues, all_dates, match_times, min_rest_days=4):
    schedule = []
    last_played = {team: start_date - timedelta(days=min_rest_days) for team in teams}
    day_info = {d: {'count': 0, 'venues': set(), 'times': set()} for d in all_dates}
    pairings = [(t1, t2) for t1 in teams for t2 in teams if t1 != t2]
    random.shuffle(pairings)
    matches_count = {(t1, t2): 0 for t1 in teams for t2 in teams if t1 != t2}
    week_dates = all_dates[::7]

    for week_start in week_dates:
        used_teams = set()
        for home, away in pairings:
            if matches_count[(home, away)] >= 1:
                continue
            if home in used_teams or away in used_teams:
                continue
            possible_days = [d for d in all_dates if week_start <= d < week_start + timedelta(days=7)
                             and (d - last_played[home]).days >= min_rest_days
                             and (d - last_played[away]).days >= min_rest_days
                             and day_info[d]['count'] < 2]
            if not possible_days:
                continue
            random.shuffle(possible_days)
            for date in possible_days:
                available_times = [t for t in match_times if t not in day_info[date]['times']]
                available_venues = [v for v in venues if v not in day_info[date]['venues']]
                if not available_times or not available_venues:
                    continue
                time = random.choice(available_times)
                venue = random.choice(available_venues)
                match = Match(home, away, date, time, venue)
                schedule.append(match)
                last_played[home] = date
                last_played[away] = date
                matches_count[(home, away)] += 1
                used_teams.update([home, away])
                day_info[date]['count'] += 1
                day_info[date]['venues'].add(venue)
                day_info[date]['times'].add(time)
                break

    schedule.sort(key=lambda m: m.date)
    return schedule

# دالة حساب اللياقة
def compute_fitness_verbose(schedule, min_rest_days=4, weights=None):
    if weights is None:
        weights = {'venue_conflict': 1.0, 'rest_violation': 0.7, 'repeated_opponent': 0.0, 'time_balance': 0.3}

    penalty = 0
    venue_day_time = {}
    last_played = {}
    matches_per_pair = {}
    team_times = {}

    venue_conflict_count = 0
    rest_violation_count = 0
    repeated_opponents_count = 0
    time_balance_penalty = 0

    for match in schedule:
        key = (match.date, match.time, match.venue)
        if key in venue_day_time:
            penalty += weights['venue_conflict']
            venue_conflict_count += 1
        else:
            venue_day_time[key] = True

        for team in [match.team1, match.team2]:
            if team in last_played:
                delta_days = (match.date - last_played[team]).days
                if delta_days < min_rest_days:
                    penalty += weights['rest_violation']
                    rest_violation_count += 1
            last_played[team] = match.date

        pair = tuple(sorted([match.team1, match.team2]))
        if pair in matches_per_pair:
            penalty += weights['repeated_opponent']
            repeated_opponents_count += 1
        matches_per_pair[pair] = matches_per_pair.get(pair,0) + 1

        for team in [match.team1, match.team2]:
            if team not in team_times:
                team_times[team] = {}
            team_times[team][match.time] = team_times[team].get(match.time,0) + 1

    for team, times in team_times.items():
        if len(times) > 1:
            diff = abs(times.get('17:00',0) - times.get('20:00',0))
            penalty += diff * weights['time_balance']
            time_balance_penalty += diff

    fitness = max(0, 100 - penalty)

    return fitness, {
        'fitness_score': fitness,
        'venue_conflicts': venue_conflict_count,
        'rest_violations': rest_violation_count,
        'repeated_opponents': repeated_opponents_count,
        'time_balance_penalty': time_balance_penalty,
        'total_penalty': penalty
    }

# ==================== إنشاء البيانات الأولية ====================

print("Creating initial test data...")

# إنشاء جداول اختبارية
baseline_schedule = generate_weekly_schedule(teams[:6], venues[:3], all_dates[:30], match_times)
parent1 = generate_weekly_schedule(teams[:6], venues[:3], all_dates[:30], match_times)
parent2 = generate_weekly_schedule(teams[:6], venues[:3], all_dates[:30], match_times)
current_schedule = baseline_schedule.copy()

# إنشاء مجموعة سكانية
population = [generate_weekly_schedule(teams[:6], venues[:3], all_dates[:30], match_times) for _ in range(10)]
fitness_scores = []
for schedule in population:
    fitness, _ = compute_fitness_verbose(schedule)
    fitness_scores.append(fitness)

print("✓ Test data created successfully!")

# ==================== واجهة المستخدم الرئيسية ====================

# إنشاء التبويبات
tab_titles = ['Genetic Operations', 'Schedule View', 'Fitness Analysis', 'About']
tabs = widgets.Tab()

# محتوى كل تبويب
genetic_operations_tab = widgets.VBox([])
schedule_view_tab = widgets.VBox([])
fitness_analysis_tab = widgets.VBox([])
about_tab = widgets.VBox([])

# تعيين التبويبات
tabs.children = [genetic_operations_tab, schedule_view_tab, fitness_analysis_tab, about_tab]
for i, title in enumerate(tab_titles):
    tabs.set_title(i, title)

# ==================== تبويب 1: العمليات الجينية ====================

# عناصر التحكم للعمليات الجينية
operation_dropdown = widgets.Dropdown(
    options=[
        ('Crossover Operation', 'crossover'),
        ('Swap Mutation', 'swap_mutation'),
        ('Change Venue Mutation', 'change_venue_mutation'),
        ('Change Time Mutation', 'change_time_mutation'),
        ('Scramble Mutation', 'scramble_mutation'),
        ('Repair Schedule', 'repair'),
        ('Tournament Selection', 'tournament_selection'),
        ('Apply All Mutations', 'all_mutations')
    ],
    value='crossover',
    description='Operation:',
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)

crossover_type_dropdown = widgets.Dropdown(
    options=[
        ('Single Point Crossover', 'single_point'),
        ('Two Point Crossover', 'two_point'),
        ('Uniform Crossover', 'uniform')
    ],
    value='single_point',
    description='Crossover Type:',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

mutation_rate_slider = widgets.FloatSlider(
    value=0.3,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Mutation Rate:',
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)

tournament_size_slider = widgets.IntSlider(
    value=3,
    min=2,
    max=10,
    step=1,
    description='Tournament Size:',
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)

num_matches_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=50,
    step=5,
    description='Matches to Show:',
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)

# أزرار التحكم
run_button = widgets.Button(
    description='Execute Operation',
    button_style='success',
    icon='play',
    layout={'width': '200px'}
)

reset_data_button = widgets.Button(
    description='Reset Test Data',
    button_style='warning',
    icon='refresh',
    layout={'width': '200px'}
)

generate_new_button = widgets.Button(
    description='Generate New Schedule',
    button_style='info',
    icon='plus',
    layout={'width': '200px'}
)

# منطقة الإخراج
genetic_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '10px'})

# دالة لتحديث عناصر التحكم
def update_genetic_controls(change=None):
    op_type = operation_dropdown.value
    
    # إظهار/إخفاء العناصر حسب العملية المختارة
    if op_type == 'crossover':
        crossover_type_dropdown.layout.visibility = 'visible'
        mutation_rate_slider.layout.visibility = 'hidden'
        tournament_size_slider.layout.visibility = 'hidden'
    elif 'mutation' in op_type:
        crossover_type_dropdown.layout.visibility = 'hidden'
        mutation_rate_slider.layout.visibility = 'visible'
        tournament_size_slider.layout.visibility = 'hidden'
    elif op_type == 'tournament_selection':
        crossover_type_dropdown.layout.visibility = 'hidden'
        mutation_rate_slider.layout.visibility = 'hidden'
        tournament_size_slider.layout.visibility = 'visible'
    else:
        crossover_type_dropdown.layout.visibility = 'hidden'
        mutation_rate_slider.layout.visibility = 'hidden'
        tournament_size_slider.layout.visibility = 'hidden'

# ربط دالة التحديث
operation_dropdown.observe(update_genetic_controls, names='value')

# دالة تنفيذ العملية الجينية
def execute_genetic_operation(b):
    with genetic_output:
        clear_output(wait=True)
        
        op_type = operation_dropdown.value
        print("=" * 70)
        print(f"EXECUTING {op_type.replace('_', ' ').upper()}")
        print("=" * 70)
        
        try:
            if op_type == 'crossover':
                child, info = apply_crossover(parent1, parent2, crossover_type_dropdown.value)
                print(f"\n Crossover completed successfully!")
                print(f"   Type: {crossover_type_dropdown.value}")
                print(f"   Child schedule created with {len(child)} matches")
                
                # تحديث الجدول الحالي
                global current_schedule
                current_schedule = child
                
                # عرض التفاصيل
                display_crossover_info(info)
                
            elif op_type == 'swap_mutation':
                mutated, info = swap_mutation(current_schedule, mutation_rate_slider.value)
                print(f"\n Swap mutation completed!")
                print(f"   Mutation rate: {mutation_rate_slider.value}")
                
                current_schedule = mutated
                display_mutation_info(info, 'Swap')
                
            elif op_type == 'change_venue_mutation':
                mutated, info = change_venue_mutation(current_schedule, venues, mutation_rate_slider.value)
                print(f"\n Change venue mutation completed!")
                
                current_schedule = mutated
                display_mutation_info(info, 'Change Venue')
                
            elif op_type == 'change_time_mutation':
                mutated, info = change_time_mutation(current_schedule, match_times, mutation_rate_slider.value)
                print(f"\n Change time mutation completed!")
                
                current_schedule = mutated
                display_mutation_info(info, 'Change Time')
                
            elif op_type == 'scramble_mutation':
                mutated, info = scramble_mutation(current_schedule, mutation_rate_slider.value)
                print(f"\n Scramble mutation completed!")
                
                current_schedule = mutated
                display_mutation_info(info, 'Scramble')
                
            elif op_type == 'repair':
                repaired, info = improved_repair_schedule(current_schedule, teams, venues, all_dates, match_times)
                print(f"\n Schedule repair completed!")
                
                current_schedule = repaired
                display_repair_info(info)
                
            elif op_type == 'tournament_selection':
                selected, info = tournament_selection(population, fitness_scores, tournament_size_slider.value)
                print(f"\n Tournament selection completed!")
                print(f"   Tournament size: {tournament_size_slider.value}")
                
                current_schedule = selected
                display_selection_info(info)
                
            elif op_type == 'all_mutations':
                mutated, info = apply_mutation(current_schedule, venues, match_times)
                print(f"\n All mutations applied!")
                
                current_schedule = mutated
                display_all_mutations_info(info)
            
            # حساب اللياقة بعد العملية
            fitness, fitness_details = compute_fitness_verbose(current_schedule)
            print(f"\n Current Schedule Fitness: {fitness:.2f}")
            print(f"   Matches: {len(current_schedule)}")
            
            # عرض عينة من المباريات
            display_schedule_sample(current_schedule)
            
        except Exception as e:
            print(f"\n Error: {e}")
            import traceback
            traceback.print_exc()

# دوال العرض المساعدة
def display_crossover_info(info):
    print(f"\n📋 Crossover Details:")
    print(f"   Parent 1 matches: {info.get('parent1_length', 'N/A')}")
    print(f"   Parent 2 matches: {info.get('parent2_length', 'N/A')}")
    if 'crossover_details' in info:
        details = info['crossover_details']
        if 'crossover_point' in details:
            print(f"   Crossover point: {details['crossover_point']}")
        print(f"   Matches from parent 1: {details.get('matches_from_parent1', 0)}")
        print(f"   Matches from parent 2: {details.get('matches_from_parent2', 0)}")
    print(f"   Final child matches: {info.get('final_child_length', 'N/A')}")

def display_mutation_info(info, mutation_type):
    print(f"\n📋 {mutation_type} Mutation Details:")
    print(f"   Mutation applied: {info.get('mutation_applied', False)}")
    if info.get('mutation_applied'):
        if 'swapped_indices' in info:
            print(f"   Swapped indices: {info['swapped_indices']}")
        if 'old_venue' in info and 'new_venue' in info:
            print(f"   Venue changed: {info['old_venue']} → {info['new_venue']}")
        if 'old_time' in info and 'new_time' in info:
            print(f"   Time changed: {info['old_time']} → {info['new_time']}")
        if 'segment_size' in info:
            print(f"   Segment shuffled: {info['segment_size']} matches")

def display_repair_info(info):
    print(f"\n📋 Repair Details:")
    print(f"   Initial matches: {info.get('initial_schedule_length', 0)}")
    print(f"   Final matches: {info.get('final_schedule_length', 0)}")
    print(f"   Success rate: {info.get('success_rate', 0)}%")
    print(f"   Venue conflicts resolved: {info.get('venue_conflicts_resolved', 0)}")
    print(f"   Team conflicts resolved: {info.get('team_conflicts_resolved', 0)}")
    print(f"   Rest periods fixed: {info.get('rest_periods_fixed', 0)}")

def display_selection_info(info):
    print(f"\n📋 Selection Details:")
    print(f"   Population size: {info.get('population_size', 0)}")
    print(f"   Winner index: {info.get('winner_index', 'N/A')}")
    print(f"   Winner fitness: {info.get('winner_fitness', 0):.2f}")
    print(f"   Tournament average fitness: {info.get('tournament_average_fitness', 0):.2f}")
    print(f"   Best in tournament: {info.get('winner_is_best_in_tournament', False)}")

def display_all_mutations_info(info):
    print(f"\n📋 All Mutations Details:")
    print(f"   Mutation types applied: {', '.join(info.get('mutation_types_applied', []))}")
    print(f"   Successful mutations: {info.get('successful_mutations_count', 0)}")
    print(f"   Schedule modified: {info.get('schedule_modified', False)}")

def display_schedule_sample(schedule, num_matches=10):
    print(f"\n📅 Schedule Sample (showing {min(num_matches, len(schedule))} matches):")
    print("-" * 80)
    print(f"{'No.':<4} {'Date':<12} {'Time':<6} {'Teams':<30} {'Venue':<20}")
    print("-" * 80)
    for i, match in enumerate(schedule[:num_matches]):
        teams_str = f"{match.team1} vs {match.team2}"
        print(f"{i+1:<4} {match.date.strftime('%Y-%m-%d'):<12} {match.time:<6} {teams_str:<30} {match.venue:<20}")

# دالة إعادة تعيين البيانات
def reset_test_data(b):
    with genetic_output:
        clear_output()
        print("Resetting test data...")
        
        global baseline_schedule, parent1, parent2, current_schedule, population, fitness_scores
        
        baseline_schedule = generate_weekly_schedule(teams[:6], venues[:3], all_dates[:30], match_times)
        parent1 = generate_weekly_schedule(teams[:6], venues[:3], all_dates[:30], match_times)
        parent2 = generate_weekly_schedule(teams[:6], venues[:3], all_dates[:30], match_times)
        current_schedule = baseline_schedule.copy()
        
        population = [generate_weekly_schedule(teams[:6], venues[:3], all_dates[:30], match_times) for _ in range(10)]
        fitness_scores = []
        for schedule in population:
            fitness, _ = compute_fitness_verbose(schedule)
            fitness_scores.append(fitness)
        
        print("✓ Test data reset successfully!")
        print(f"   - Current schedule: {len(current_schedule)} matches")
        print(f"   - Population: {len(population)} schedules")
        print(f"   - Fitness scores range: {min(fitness_scores):.2f} - {max(fitness_scores):.2f}")

# دالة إنشاء جدول جديد
def generate_new_schedule(b):
    with genetic_output:
        clear_output()
        print("Generating new schedule...")
        
        global current_schedule
        current_schedule = generate_weekly_schedule(teams[:8], venues[:4], all_dates[:60], match_times)
        
        fitness, details = compute_fitness_verbose(current_schedule)
        print("✓ New schedule generated successfully!")
        print(f"   - Matches: {len(current_schedule)}")
        print(f"   - Fitness score: {fitness:.2f}")
        print(f"   - Venue conflicts: {details['venue_conflicts']}")
        print(f"   - Rest violations: {details['rest_violations']}")
        
        display_schedule_sample(current_schedule)

# ربط الأزرار بالدوال
run_button.on_click(execute_genetic_operation)
reset_data_button.on_click(reset_test_data)
generate_new_button.on_click(generate_new_schedule)

# تحديث العناصر أول مرة
update_genetic_controls()

# تجميع عناصر تبويب العمليات الجينية
genetic_controls = widgets.VBox([
    widgets.HTML("<h3>Genetic Algorithm Operations</h3>"),
    operation_dropdown,
    crossover_type_dropdown,
    mutation_rate_slider,
    tournament_size_slider,
    widgets.HTML("<br>"),
    widgets.HBox([run_button, reset_data_button, generate_new_button])
])

genetic_operations_tab.children = [genetic_controls, genetic_output]

# ==================== تبويب 2: عرض الجداول ====================

# عناصر التحكم لعرض الجداول
view_type_dropdown = widgets.Dropdown(
    options=[
        ('Current Schedule', 'current'),
        ('Baseline Schedule', 'baseline'),
        ('Parent 1', 'parent1'),
        ('Parent 2', 'parent2'),
        ('Population Best', 'best')
    ],
    value='current',
    description='View:',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

team_dropdown = widgets.Dropdown(
    options=teams,
    value=teams[0],
    description='Team:',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

view_button = widgets.Button(
    description='View Schedule',
    button_style='primary',
    layout={'width': '150px'}
)

view_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '10px'})

def view_schedule(b):
    with view_output:
        clear_output(wait=True)
        
        view_type = view_type_dropdown.value
        
        if view_type == 'current':
            schedule = current_schedule
            title = "Current Schedule"
        elif view_type == 'baseline':
            schedule = baseline_schedule
            title = "Baseline Schedule"
        elif view_type == 'parent1':
            schedule = parent1
            title = "Parent 1 Schedule"
        elif view_type == 'parent2':
            schedule = parent2
            title = "Parent 2 Schedule"
        elif view_type == 'best':
            best_idx = fitness_scores.index(max(fitness_scores))
            schedule = population[best_idx]
            title = f"Best Schedule (Fitness: {fitness_scores[best_idx]:.2f})"
        
        print(f"📋 {title}")
        print("=" * 80)
        print(f"Total matches: {len(schedule)}")
        
        # حساب اللياقة
        fitness, details = compute_fitness_verbose(schedule)
        print(f"\n📊 Fitness Analysis:")
        print(f"   Fitness score: {fitness:.2f}")
        print(f"   Venue conflicts: {details['venue_conflicts']}")
        print(f"   Rest violations: {details['rest_violations']}")
        print(f"   Repeated opponents: {details['repeated_opponents']}")
        print(f"   Time balance penalty: {details['time_balance_penalty']}")
        
        # عرض المباريات
        display_schedule_sample(schedule, num_matches_slider.value)
        
        # عرض جدول فريق معين
        print(f"\n👥 Schedule for {team_dropdown.value}:")
        team_matches = [m for m in schedule if m.team1 == team_dropdown.value or m.team2 == team_dropdown.value]
        team_matches.sort(key=lambda m: m.date)
        
        if team_matches:
            print(f"   Total matches: {len(team_matches)}")
            for i, match in enumerate(team_matches[:10]):
                opponent = match.team2 if match.team1 == team_dropdown.value else match.team1
                home_away = "Home" if match.team1 == team_dropdown.value else "Away"
                print(f"   {i+1}. {match.date.strftime('%Y-%m-%d')} {match.time} vs {opponent} ({home_away}) at {match.venue}")
        else:
            print("   No matches found for this team.")

view_button.on_click(view_schedule)

# تجميع عناصر تبويب العرض
schedule_controls = widgets.VBox([
    widgets.HTML("<h3>Schedule Viewer</h3>"),
    view_type_dropdown,
    team_dropdown,
    num_matches_slider,
    view_button
])

schedule_view_tab.children = [schedule_controls, view_output]

# ==================== تبويب 3: تحليل اللياقة ====================

fitness_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '10px'})

analyze_button = widgets.Button(
    description='Analyze Fitness',
    button_style='info',
    layout={'width': '150px'}
)

def analyze_fitness(b):
    with fitness_output:
        clear_output(wait=True)
        
        print("📊 FITNESS ANALYSIS REPORT")
        print("=" * 80)
        
        # تحليل الجدول الحالي
        fitness_current, details_current = compute_fitness_verbose(current_schedule)
        fitness_baseline, details_baseline = compute_fitness_verbose(baseline_schedule)
        
        print("\n1. CURRENT SCHEDULE:")
        print(f"   Fitness: {fitness_current:.2f}")
        print(f"   Matches: {len(current_schedule)}")
        print(f"   Venue conflicts: {details_current['venue_conflicts']}")
        print(f"   Rest violations: {details_current['rest_violations']}")
        
        print("\n2. BASELINE SCHEDULE:")
        print(f"   Fitness: {fitness_baseline:.2f}")
        print(f"   Matches: {len(baseline_schedule)}")
        
        print(f"\n3. IMPROVEMENT: {fitness_current - fitness_baseline:+.2f} points")
        
        # تحليل السكان
        print("\n4. POPULATION ANALYSIS:")
        print(f"   Population size: {len(population)}")
        print(f"   Best fitness: {max(fitness_scores):.2f}")
        print(f"   Worst fitness: {min(fitness_scores):.2f}")
        print(f"   Average fitness: {sum(fitness_scores)/len(fitness_scores):.2f}")
        
        # رسم بياني للياقة
        plt.figure(figsize=(10, 6))
        
        # البيانات للرسم
        labels = ['Current', 'Baseline', 'Best Pop', 'Worst Pop', 'Avg Pop']
        values = [
            fitness_current,
            fitness_baseline,
            max(fitness_scores),
            min(fitness_scores),
            sum(fitness_scores)/len(fitness_scores)
        ]
        
        colors = ['green', 'blue', 'gold', 'red', 'orange']
        
        bars = plt.bar(labels, values, color=colors)
        plt.axhline(y=70, color='r', linestyle='--', alpha=0.3, label='Good Fitness (70)')
        plt.axhline(y=85, color='g', linestyle='--', alpha=0.3, label='Excellent Fitness (85)')
        
        plt.ylabel('Fitness Score')
        plt.title('Fitness Comparison')
        plt.ylim(0, 100)
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        # إضافة القيم على الأعمدة
        for bar, value in zip(bars, values):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    f'{value:.1f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        # تحليل المفصل للجدول الحالي
        print("\n5. DETAILED CURRENT SCHEDULE ANALYSIS:")
        
        # تحليل التوزيع
        venues_used = {}
        dates_used = {}
        teams_count = {}
        
        for match in current_schedule:
            venues_used[match.venue] = venues_used.get(match.venue, 0) + 1
            date_str = match.date.strftime('%Y-%m-%d')
            dates_used[date_str] = dates_used.get(date_str, 0) + 1
            teams_count[match.team1] = teams_count.get(match.team1, 0) + 1
            teams_count[match.team2] = teams_count.get(match.team2, 0) + 1
        
        print(f"   Venues used: {len(venues_used)}")
        print(f"   Dates with matches: {len(dates_used)}")
        print(f"   Teams in schedule: {len(teams_count)}")
        
        # أكثر الملاعب استخداماً
        print("\n   Most used venues:")
        sorted_venues = sorted(venues_used.items(), key=lambda x: x[1], reverse=True)[:3]
        for venue, count in sorted_venues:
            print(f"     {venue}: {count} matches")

analyze_button.on_click(analyze_fitness)

# تجميع عناصر تبويب اللياقة
fitness_analysis_tab.children = [
    widgets.VBox([
        widgets.HTML("<h3>Fitness Analysis</h3>"),
        analyze_button
    ]),
    fitness_output
]

# ==================== تبويب 4: معلومات عن المشروع ====================

about_content = widgets.VBox([
    widgets.HTML("""
    <div style="padding: 20px;">
        <h2>🎯 Sports Tournament Scheduling System</h2>
        <p><strong>Genetic Algorithm Implementation for Optimal Schedule Generation</strong></p>
        
        <h3>📋 Features:</h3>
        <ul>
            <li><strong>Crossover Operations:</strong> Single-point, Two-point, Uniform crossover</li>
            <li><strong>Mutation Operations:</strong> Swap, Change Venue, Change Time, Scramble</li>
            <li><strong>Selection:</strong> Tournament selection for parent selection</li>
            <li><strong>Repair Mechanism:</strong> Automatic schedule repair for constraints</li>
            <li><strong>Fitness Evaluation:</strong> Comprehensive fitness scoring system</li>
            <li><strong>Visual Analysis:</strong> Graphical fitness comparison</li>
        </ul>
        
        <h3>⚙️ Technical Details:</h3>
        <ul>
            <li><strong>Teams:</strong> 18 Egyptian football teams</li>
            <li><strong>Venues:</strong> 10 stadiums across Egypt</li>
            <li><strong>Time slots:</strong> 17:00 and 20:00</li>
            <li><strong>Tournament period:</strong> May 2025 - January 2026</li>
            <li><strong>Minimum rest days:</strong> 4 days between matches</li>
        </ul>
        
        <h3>📊 Fitness Function Components:</h3>
        <ul>
            <li>Venue conflict penalty: 1.0 point per conflict</li>
            <li>Rest violation penalty: 0.7 points per violation</li>
            <li>Time balance penalty: 0.3 points per imbalance</li>
            <li>Maximum possible fitness: 100 points</li>
        </ul>
        
        <p><strong>🎯 Goal:</strong> Generate optimal tournament schedules that maximize fitness while respecting all constraints.</p>
    </div>
    """)
])

about_tab.children = [about_content]

# ==================== عرض الواجهة الرئيسية ====================

print("=" * 80)
print("SPORTS TOURNAMENT SCHEDULING SYSTEM - GENETIC ALGORITHM GUI")
print("=" * 80)
print(" All genetic algorithm functions loaded successfully!")
print(f" Test data initialized with {len(current_schedule)} matches")
print(f" Population created with {len(population)} schedules")
print(" GUI ready for operations")
print("=" * 80)
print("\nNavigate through tabs to use different features:\n")
print("1. Genetic Operations - Execute crossover, mutation, and selection")
print("2. Schedule View - View and analyze different schedules")
print("3. Fitness Analysis - Comprehensive fitness evaluation")
print("4. About - System information and documentation")
print("\n" + "=" * 80)

# عرض الواجهة
display(tabs)

# عرض حالة النظام الحالية
print("\n📈 CURRENT SYSTEM STATUS:")
print(f"   • Current schedule fitness: {compute_fitness_verbose(current_schedule)[0]:.2f}")
print(f"   • Population best fitness: {max(fitness_scores):.2f}")
print(f"   • Population average fitness: {sum(fitness_scores)/len(fitness_scores):.2f}")
print(f"   • Available teams: {len(teams)}")
print(f"   • Available venues: {len(venues)}")
print(f"   • Available dates: {len(all_dates)}")
print("=" * 80)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# ==================== Cell 1: Imports and Setup ====================
import random
import copy
import json
import csv
from datetime import datetime, timedelta
from typing import List, Tuple, Dict, Any
import sys
import os

# Add parent directory to path for imports
sys.path.append('../')

# Import shared modules - Task 1, 2, 3
# جميع المهام متكاملة في data package
from data.models import Match
from data.fitness import compute_fitness
from data.teams_venues_times import teams, venues, match_times
from data.schedule_generator import generate_weekly_schedule  # Task 1
from data.genetic_operations import (  # Task 3
    single_point_crossover,
    two_point_crossover,
    uniform_crossover,
    swap_mutation,
    change_venue_mutation,
    change_time_mutation,
    tournament_selection,
    roulette_wheel_selection,
    rank_selection
)

# Tournament period
start_date = datetime(2025, 5, 1)
end_date = datetime(2026, 1, 31)

# Setup all dates
all_dates = []
current_date = start_date
while current_date <= end_date:
    all_dates.append(current_date)
    current_date += timedelta(days=1)

print("=" * 80)
print("✅ All imports successful!")
print("=" * 80)
print(f"📋 Teams: {len(teams)}")
print(f"🏟️  Venues: {len(venues)}")
print(f"⏰ Match times: {match_times}")
print(f"📅 Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"📆 Total days: {len(all_dates)}")
print("\n🔍 Integration Status:")
print(f"   ✅ Task 1: schedule_generator - {generate_weekly_schedule.__name__}")
print(f"   ✅ Task 2: fitness - {compute_fitness.__name__}")
print(f"   ✅ Task 3: genetic_operations - {single_point_crossover.__name__}, {tournament_selection.__name__}")
print("=" * 80)


In [ ]:
# ==================== Cell 2: Validation Functions ====================
def validate_schedule(schedule, teams, min_rest_days=4):
    """
    Validate schedule constraints with comprehensive checks
    
    Returns:
    --------
    tuple: (is_valid: bool, errors: List[str])
    """
    errors = []
    expected_matches = len(teams) * (len(teams) - 1)  # 18 × 17 = 306
    
    # Check total match count
    if len(schedule) != expected_matches:
        errors.append(f"Expected {expected_matches} matches, got {len(schedule)}")
    
    # Check for duplicate matches (same teams, same date)
    seen_matches = {}
    for match in schedule:
        key = (match.team1, match.team2, match.date)
        if key in seen_matches:
            errors.append(f"Duplicate match: {match}")
        seen_matches[key] = True
    
    # Check for self-play
    for match in schedule:
        if match.team1 == match.team2:
            errors.append(f"Team playing itself: {match}")
    
    # Check match count per team (should be 34: 17 opponents × 2)
    team_match_count = {team: 0 for team in teams}
    for match in schedule:
        team_match_count[match.team1] += 1
        team_match_count[match.team2] += 1
    
    expected_per_team = (len(teams) - 1) * 2  # 17 × 2 = 34
    for team in teams:
        if team_match_count.get(team, 0) != expected_per_team:
            errors.append(f"Team {team} has {team_match_count.get(team, 0)} matches, expected {expected_per_team}")
    
    # Check home/away balance for each pair
    pair_matches = {}
    for match in schedule:
        pair = tuple(sorted([match.team1, match.team2]))
        if pair not in pair_matches:
            pair_matches[pair] = {'home': 0, 'away': 0}
        
        # Determine if team1 is home or away based on pair order
        if match.team1 == pair[0]:
            pair_matches[pair]['home'] += 1
        else:
            pair_matches[pair]['away'] += 1
    
    for pair, counts in pair_matches.items():
        if counts['home'] != 1 or counts['away'] != 1:
            errors.append(f"Pair {pair} has unbalanced home/away: home={counts['home']}, away={counts['away']} (expected 1 each)")
    
    # Check rest days
    last_played = {}
    for match in schedule:
        for team in [match.team1, match.team2]:
            if team in last_played:
                delta = (match.date - last_played[team]).days
                if delta < min_rest_days:
                    errors.append(f"Rest violation: {team} played {delta} days apart (min {min_rest_days} required)")
            last_played[team] = match.date
    
    # Check venue conflicts
    venue_slots = {}
    for match in schedule:
        key = (match.date, match.time, match.venue)
        if key in venue_slots:
            errors.append(f"Venue conflict: {match}")
        venue_slots[key] = True
    
    # Check consecutive home matches (no team should have 3+ consecutive home matches)
    team_schedule = {team: [] for team in teams}
    for match in schedule:
        team_schedule[match.team1].append(('home', match.date))
        team_schedule[match.team2].append(('away', match.date))
    
    for team, matches in team_schedule.items():
        matches.sort(key=lambda x: x[1])
        consecutive_home = 0
        for match_type, date in matches:
            if match_type == 'home':
                consecutive_home += 1
                if consecutive_home >= 3:
                    errors.append(f"Team {team} has {consecutive_home} consecutive home matches")
                    break
            else:
                consecutive_home = 0
    
    return len(errors) == 0, errors

print("✅ Validation functions loaded!")


In [ ]:
# ==================== Cell 3: Repair Functions ====================
def regenerate_missing_matches(schedule, teams, venues, match_times, all_dates, min_rest_days=4):
    """Regenerate missing matches to reach 306 matches"""
    expected_matches = len(teams) * (len(teams) - 1)  # 306
    current_count = len(schedule)
    
    if current_count >= expected_matches:
        return schedule
    
    # Track existing matches
    existing_pairs = set()
    for match in schedule:
        existing_pairs.add((match.team1, match.team2))
    
    # Track venue/time usage per day
    day_info = {}
    for match in schedule:
        if match.date not in day_info:
            day_info[match.date] = {'count': 0, 'venues': set(), 'times': set()}
        day_info[match.date]['count'] += 1
        day_info[match.date]['venues'].add(match.venue)
        day_info[match.date]['times'].add(match.time)
    
    # Find missing pairings
    missing_pairings = []
    for t1 in teams:
        for t2 in teams:
            if t1 != t2 and (t1, t2) not in existing_pairs:
                missing_pairings.append((t1, t2))
    
    random.shuffle(missing_pairings)
    
    # Try to schedule missing matches
    for home, away in missing_pairings:
        if len(schedule) >= expected_matches:
            break
        
        for date in all_dates:
            if len(schedule) >= expected_matches:
                break
            
            if date not in day_info:
                day_info[date] = {'count': 0, 'venues': set(), 'times': set()}
            
            if day_info[date]['count'] < 2:  # Max 2 matches per day
                available_times = [t for t in match_times if t not in day_info[date]['times']]
                available_venues = [v for v in venues if v not in day_info[date]['venues']]
                
                if available_times and available_venues:
                    time = random.choice(available_times)
                    venue = random.choice(available_venues)
                    match = Match(home, away, date, time, venue)
                    schedule.append(match)
                    day_info[date]['count'] += 1
                    day_info[date]['venues'].add(venue)
                    day_info[date]['times'].add(time)
                    break
    
    return schedule


def repair_schedule(schedule, teams, venues, match_times, all_dates, min_rest_days=4, max_retries=5):
    """
    Repair invalid schedule by removing duplicates and fixing conflicts
    Includes retry mechanism and match regeneration
    """
    best_repaired = schedule
    best_match_count = len(schedule)
    expected_matches = len(teams) * (len(teams) - 1)  # 306
    
    for retry in range(max_retries):
        # Remove duplicates and self-play matches
        seen = set()
        valid_schedule = []
        for match in schedule:
            key = (match.team1, match.team2, match.date)
            if key not in seen and match.team1 != match.team2:
                seen.add(key)
                valid_schedule.append(copy.deepcopy(match))
        
        # Remove venue conflicts (keep first occurrence)
        venue_slots = {}
        repaired = []
        for match in valid_schedule:
            key = (match.date, match.time, match.venue)
            if key not in venue_slots:
                venue_slots[key] = True
                repaired.append(match)
            else:
                # Try to fix by changing venue or time
                available_venues = [v for v in venues if v != match.venue]
                available_times = [t for t in match_times if t != match.time]
                
                fixed = False
                # Try changing venue first
                for new_venue in available_venues:
                    new_key = (match.date, match.time, new_venue)
                    if new_key not in venue_slots:
                        match.venue = new_venue
                        venue_slots[new_key] = True
                        repaired.append(match)
                        fixed = True
                        break
                
                # If venue change didn't work, try changing time
                if not fixed:
                    for new_time in available_times:
                        new_key = (match.date, new_time, match.venue)
                        if new_key not in venue_slots:
                            match.time = new_time
                            venue_slots[new_key] = True
                            repaired.append(match)
                            fixed = True
                            break
                
                # If still not fixed, try changing both venue and time
                if not fixed:
                    for new_venue in available_venues:
                        for new_time in available_times:
                            new_key = (match.date, new_time, new_venue)
                            if new_key not in venue_slots:
                                match.venue = new_venue
                                match.time = new_time
                                venue_slots[new_key] = True
                                repaired.append(match)
                                fixed = True
                                break
                        if fixed:
                            break
        
        # If we lost too many matches, try to regenerate
        if len(repaired) < expected_matches * 0.9:  # Lost more than 10%
            repaired = regenerate_missing_matches(repaired, teams, venues, match_times, all_dates, min_rest_days)
        
        # Keep track of best repair attempt
        if len(repaired) > best_match_count:
            best_repaired = repaired
            best_match_count = len(repaired)
        
        # If we have enough matches and it's valid, return
        if len(repaired) >= expected_matches * 0.95:  # At least 95% of expected
            repaired.sort(key=lambda m: m.date)
            return repaired
        
        # For next retry, use the repaired schedule as base
        schedule = repaired
    
    # Return best attempt
    best_repaired.sort(key=lambda m: m.date)
    return best_repaired

print("✅ Repair functions loaded!")


In [ ]:
# ==================== Cell 4: Mutation Wrapper ====================
def apply_mutation(schedule, venues, match_times, mutation_rate=0.1):
    """
    Apply mutation based on mutation_rate
    Uses mutation functions from Task 3 (genetic_operations)
    """
    # Choose mutation type (can be weighted)
    mutation_type = random.choice(['swap', 'venue', 'time'])
    
    if mutation_type == 'swap':
        return swap_mutation(schedule, mutation_rate)
    elif mutation_type == 'venue':
        return change_venue_mutation(schedule, venues, mutation_rate)
    elif mutation_type == 'time':
        return change_time_mutation(schedule, match_times, mutation_rate)
    
    return copy.deepcopy(schedule)

print("✅ Mutation wrapper loaded!")


In [ ]:
# ==================== Cell 5: Genetic Algorithm Main Loop ====================
def run_genetic_algorithm(
    population_size=50,
    generations=100,
    mutation_rate=0.1,
    crossover_rate=0.8,
    elitism_count=2,
    selection_method='tournament',
    tournament_size=3,
    crossover_method='single_point',
    verbose=True
):
    """
    Run the complete genetic algorithm
    
    Parameters:
    -----------
    population_size : int
        Number of individuals in the population
    generations : int
        Number of generations to evolve
    mutation_rate : float
        Probability of mutation (0.0 to 1.0)
    crossover_rate : float
        Probability of crossover (0.0 to 1.0)
    elitism_count : int
        Number of best individuals to preserve
    selection_method : str
        'tournament', 'roulette', or 'rank'
    tournament_size : int
        Size of tournament for tournament selection
    crossover_method : str
        'single_point', 'two_point', or 'uniform'
    verbose : bool
        Print progress information
    
    Returns:
    --------
    best_schedule : List[Match]
        Best schedule found
    best_fitness : float
        Fitness of best schedule
    history : Dict
        Evolution history with fitness statistics
    """
    
    # Initialize population
    if verbose:
        print(f"Initializing population of {population_size} schedules...")
    
    population = [generate_weekly_schedule(teams, venues, all_dates, match_times, min_rest_days=4, start_date=start_date) 
                  for _ in range(population_size)]
    
    # Evaluate initial population
    fitness_scores = [compute_fitness(schedule) for schedule in population]
    
    # Track history
    history = {
        'best_fitness': [],
        'avg_fitness': [],
        'worst_fitness': [],
        'generation': []
    }
    
    best_fitness = max(fitness_scores)
    best_schedule = copy.deepcopy(population[fitness_scores.index(best_fitness)])
    
    if verbose:
        print(f"Initial best fitness: {best_fitness:.2f}")
    
    # Evolution loop
    for generation in range(generations):
        # Record statistics
        history['generation'].append(generation)
        history['best_fitness'].append(best_fitness)
        history['avg_fitness'].append(sum(fitness_scores) / len(fitness_scores))
        history['worst_fitness'].append(min(fitness_scores))
        
        # Create new population
        new_population = []
        
        # Elitism: keep best individuals
        elite_indices = sorted(range(len(fitness_scores)), 
                               key=lambda i: fitness_scores[i], 
                               reverse=True)[:elitism_count]
        for idx in elite_indices:
            new_population.append(copy.deepcopy(population[idx]))
        
        # Generate remaining population
        while len(new_population) < population_size:
            # Selection
            if selection_method == 'tournament':
                parent1 = tournament_selection(population, fitness_scores, tournament_size)
                parent2 = tournament_selection(population, fitness_scores, tournament_size)
            elif selection_method == 'roulette':
                parent1 = roulette_wheel_selection(population, fitness_scores)
                parent2 = roulette_wheel_selection(population, fitness_scores)
            elif selection_method == 'rank':
                parent1 = rank_selection(population, fitness_scores)
                parent2 = rank_selection(population, fitness_scores)
            else:
                parent1 = tournament_selection(population, fitness_scores, tournament_size)
                parent2 = tournament_selection(population, fitness_scores, tournament_size)
            
            # Crossover
            if random.random() < crossover_rate:
                if crossover_method == 'two_point':
                    child = two_point_crossover(parent1, parent2)
                elif crossover_method == 'uniform':
                    child = uniform_crossover(parent1, parent2)
                else:
                    child = single_point_crossover(parent1, parent2)
            else:
                child = copy.deepcopy(parent1 if random.random() < 0.5 else parent2)
            
            # Mutation
            child = apply_mutation(child, venues, match_times, mutation_rate)
            
            # Validation and repair
            is_valid, errors = validate_schedule(child, teams, min_rest_days=4)
            if not is_valid:
                child = repair_schedule(child, teams, venues, match_times, all_dates, min_rest_days=4)
                # If still invalid or lost too many matches, use parent
                if len(child) < len(parent1) * 0.8:  # Lost too many matches
                    child = copy.deepcopy(parent1)
            
            new_population.append(child)
        
        # Update population
        population = new_population
        fitness_scores = [compute_fitness(schedule) for schedule in population]
        
        # Update best
        current_best = max(fitness_scores)
        if current_best > best_fitness:
            best_fitness = current_best
            best_schedule = copy.deepcopy(population[fitness_scores.index(current_best)])
        
        if verbose and (generation + 1) % 10 == 0:
            print(f"Generation {generation + 1}/{generations}: "
                  f"Best={best_fitness:.2f}, "
                  f"Avg={history['avg_fitness'][-1]:.2f}, "
                  f"Worst={history['worst_fitness'][-1]:.2f}")
    
    if verbose:
        print(f"\nFinal best fitness: {best_fitness:.2f}")
    
    return best_schedule, best_fitness, history

print("✅ Genetic Algorithm function loaded!")


In [ ]:
# ==================== Cell 6: Experiment Functions ====================
def run_experiments(experiment_configs, num_runs=3):
    """
    Run multiple experiments with different parameter configurations
    
    Parameters:
    -----------
    experiment_configs : List[Dict]
        List of parameter configurations to test
    num_runs : int
        Number of runs per configuration for averaging
    
    Returns:
    --------
    results : List[Dict]
        Results for each experiment configuration
    """
    results = []
    
    print("=" * 80)
    print("GENETIC ALGORITHM EXPERIMENTS")
    print("=" * 80)
    
    for exp_idx, config in enumerate(experiment_configs):
        print(f"\n{'='*80}")
        print(f"Experiment {exp_idx + 1}/{len(experiment_configs)}")
        print(f"Configuration: {config}")
        print(f"{'='*80}\n")
        
        run_results = []
        
        for run in range(num_runs):
            print(f"Run {run + 1}/{num_runs}...", end=" ")
            
            best_schedule, best_fitness, history = run_genetic_algorithm(
                population_size=config.get('population_size', 50),
                generations=config.get('generations', 100),
                mutation_rate=config.get('mutation_rate', 0.1),
                crossover_rate=config.get('crossover_rate', 0.8),
                elitism_count=config.get('elitism_count', 2),
                selection_method=config.get('selection_method', 'tournament'),
                tournament_size=config.get('tournament_size', 3),
                crossover_method=config.get('crossover_method', 'single_point'),
                verbose=False
            )
            
            run_results.append({
                'best_fitness': best_fitness,
                'final_avg_fitness': history['avg_fitness'][-1],
                'final_worst_fitness': history['worst_fitness'][-1],
                'history': history
            })
            
            print(f"Best fitness: {best_fitness:.2f}")
        
        # Calculate statistics
        best_fitnesses = [r['best_fitness'] for r in run_results]
        avg_fitnesses = [r['final_avg_fitness'] for r in run_results]
        
        result = {
            'config': config,
            'runs': run_results,
            'statistics': {
                'best_fitness_mean': sum(best_fitnesses) / len(best_fitnesses),
                'best_fitness_std': (sum((x - sum(best_fitnesses)/len(best_fitnesses))**2 
                                     for x in best_fitnesses) / len(best_fitnesses))**0.5,
                'best_fitness_max': max(best_fitnesses),
                'best_fitness_min': min(best_fitnesses),
                'avg_fitness_mean': sum(avg_fitnesses) / len(avg_fitnesses),
            }
        }
        
        results.append(result)
        
        print(f"\nStatistics:")
        print(f"  Best Fitness - Mean: {result['statistics']['best_fitness_mean']:.2f}, "
              f"Std: {result['statistics']['best_fitness_std']:.2f}, "
              f"Max: {result['statistics']['best_fitness_max']:.2f}, "
              f"Min: {result['statistics']['best_fitness_min']:.2f}")
    
    return results


def save_results(results, filename='experiment_results.json'):
    """Save experiment results to JSON file"""
    # Convert results to JSON-serializable format
    json_results = []
    for result in results:
        json_result = {
            'config': result['config'],
            'statistics': result['statistics'],
            'runs': []
        }
        # Only save final statistics for each run (not full history)
        for run in result['runs']:
            json_result['runs'].append({
                'best_fitness': run['best_fitness'],
                'final_avg_fitness': run['final_avg_fitness'],
                'final_worst_fitness': run['final_worst_fitness']
            })
        json_results.append(json_result)
    
    with open(filename, 'w') as f:
        json.dump(json_results, f, indent=2)
    print(f"\nResults saved to {filename}")


def save_results_csv(results, filename='experiment_results.csv'):
    """Save experiment results summary to CSV"""
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            'Experiment', 'Population Size', 'Generations', 'Mutation Rate',
            'Crossover Rate', 'Elitism Count', 'Selection Method',
            'Tournament Size', 'Crossover Method',
            'Best Fitness Mean', 'Best Fitness Std', 'Best Fitness Max', 'Best Fitness Min',
            'Avg Fitness Mean'
        ])
        
        for idx, result in enumerate(results):
            config = result['config']
            stats = result['statistics']
            writer.writerow([
                idx + 1,
                config.get('population_size', 'N/A'),
                config.get('generations', 'N/A'),
                config.get('mutation_rate', 'N/A'),
                config.get('crossover_rate', 'N/A'),
                config.get('elitism_count', 'N/A'),
                config.get('selection_method', 'N/A'),
                config.get('tournament_size', 'N/A'),
                config.get('crossover_method', 'N/A'),
                f"{stats['best_fitness_mean']:.2f}",
                f"{stats['best_fitness_std']:.2f}",
                f"{stats['best_fitness_max']:.2f}",
                f"{stats['best_fitness_min']:.2f}",
                f"{stats['avg_fitness_mean']:.2f}"
            ])
    
    print(f"Results summary saved to {filename}")


def compare_results(results):
    """Print a comparison of all experiment results"""
    print("\n" + "=" * 80)
    print("EXPERIMENT RESULTS COMPARISON")
    print("=" * 80)
    
    # Sort by best fitness mean
    sorted_results = sorted(results, 
                           key=lambda x: x['statistics']['best_fitness_mean'], 
                           reverse=True)
    
    print(f"\n{'Exp':<5} {'Pop':<5} {'Gen':<5} {'Mut':<6} {'Sel':<12} {'Best Mean':<12} {'Best Max':<12} {'Best Min':<12}")
    print("-" * 80)
    
    for idx, result in enumerate(sorted_results):
        config = result['config']
        stats = result['statistics']
        print(f"{idx+1:<5} "
              f"{config.get('population_size', 'N/A'):<5} "
              f"{config.get('generations', 'N/A'):<5} "
              f"{config.get('mutation_rate', 'N/A'):<6.2f} "
              f"{config.get('selection_method', 'N/A'):<12} "
              f"{stats['best_fitness_mean']:<12.2f} "
              f"{stats['best_fitness_max']:<12.2f} "
              f"{stats['best_fitness_min']:<12.2f}")
    
    print("\n" + "=" * 80)
    print(f"Best Configuration: Experiment {sorted_results[0]['config']}")
    print(f"Best Fitness Achieved: {sorted_results[0]['statistics']['best_fitness_max']:.2f}")
    print("=" * 80)

print("✅ Experiment functions loaded!")


In [ ]:
# Cell 7: Test Run
# Test with small parameters
print("Running test GA...")
best_schedule, best_fitness, history = run_genetic_algorithm(
    population_size=10,
    generations=5,
    mutation_rate=0.1,
    verbose=True
)

print(f"\n✅ Test completed!")
print(f"Best schedule has {len(best_schedule)} matches")
print(f"Best fitness: {best_fitness:.2f}")

# Validate
is_valid, errors = validate_schedule(best_schedule, teams)
if is_valid:
    print("✅ Schedule is valid!")
else:
    print(f"⚠️ Schedule has {len(errors)} validation errors")
    if len(errors) <= 5:
        for error in errors:
            print(f"  - {error}")


In [ ]:
# ==================== Cell 8: Run Full Experiments ====================
# Define experiment configurations (reduced from 11 to 6 experiments)
experiments = [
    # Baseline
    {
        'name': 'Baseline',
        'population_size': 30,
        'generations': 50,
        'mutation_rate': 0.1,
        'crossover_rate': 0.8,
        'elitism_count': 2,
        'selection_method': 'tournament',
        'tournament_size': 3,
        'crossover_method': 'single_point'
    },
    # Test population sizes
    {
        'name': 'Large Population',
        'population_size': 100,
        'generations': 50,
        'mutation_rate': 0.1,
        'crossover_rate': 0.8,
        'elitism_count': 2,
        'selection_method': 'tournament',
        'tournament_size': 3,
        'crossover_method': 'single_point'
    },
    # Test mutation rates
    {
        'name': 'High Mutation',
        'population_size': 50,
        'generations': 50,
        'mutation_rate': 0.3,
        'crossover_rate': 0.8,
        'elitism_count': 2,
        'selection_method': 'tournament',
        'tournament_size': 3,
        'crossover_method': 'single_point'
    },
    # Test selection methods
    {
        'name': 'Roulette Selection',
        'population_size': 50,
        'generations': 50,
        'mutation_rate': 0.1,
        'crossover_rate': 0.8,
        'elitism_count': 2,
        'selection_method': 'roulette',
        'tournament_size': 3,
        'crossover_method': 'single_point'
    },
    # Test crossover methods
    {
        'name': 'Two-Point Crossover',
        'population_size': 50,
        'generations': 50,
        'mutation_rate': 0.1,
        'crossover_rate': 0.8,
        'elitism_count': 2,
        'selection_method': 'tournament',
        'tournament_size': 3,
        'crossover_method': 'two_point'
    },
    # Best combination
    {
        'name': 'Best Combination',
        'population_size': 100,
        'generations': 100,
        'mutation_rate': 0.15,
        'crossover_rate': 0.8,
        'elitism_count': 5,
        'selection_method': 'tournament',
        'tournament_size': 5,
        'crossover_method': 'two_point'
    }
]

# Run experiments (with fewer runs for faster execution)
print("Starting experiments...")
results = run_experiments(experiments, num_runs=3)

# Compare and save results
compare_results(results)
save_results(results, 'experiment_results.json')
save_results_csv(results, 'experiment_results.csv')

print("\n✅ All experiments completed!")
